In [1]:
import numpy as np
import pandas as pd
import cobra.test
import os

from os.path import join

from time import time


from cobra.flux_analysis import (
    single_gene_deletion, single_reaction_deletion, double_gene_deletion,
    double_reaction_deletion)

import cobra
from cobra.test import create_test_model
cobra_config = cobra.Configuration()
cobra_config.solver = "gurobi"
model = create_test_model("textbook")
model.solver

a = cobra.io.read_sbml_model("iEC1344_C.xml")


Academic license - for non-commercial use only - expires 2024-03-07
Using license file /home/schwartzlab/gurobi.lic


In [2]:
int_reactions= list()
i= -1
n= 0
r = a.reactions[0]
r_id = r.id
zero= list()
one= list()
one_at_zero= list()

for r in a.reactions:  #finds reactions that have one feasible value at flux=0 or for another flux
    i= i+1
    r = a.reactions[i]
    r_id = r.id

    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
    v= len(csvFile["Biomass"])-csvFile["Biomass"].isna().sum()
    n= int((len(csvFile)-1)/2)

    flt= float(csvFile.iloc[n,  2])
    
    if v == 1 and csvFile.iloc[n,  2] != flt:
        one.append(r_id)
    elif v == 0:
        zero.append(r_id)
        
    if v == 1 and csvFile.iloc[n,  2] == flt:
        one_at_zero.append(r_id)

print(one)
print(zero)
print(one_at_zero)

D= pd.DataFrame (one, columns = ['1 feasible value'])
D_at_zero= pd.DataFrame (one_at_zero, columns = ['1 feasible value_at_zero'])


[]
[]
['EX_cm_e', 'EX_cobalt2_e', 'EX_colipa_e', 'EX_colipap_e', 'EX_crn_e', 'EX_crn__D_e', 'DM_4crsol_c', 'EX_cu2_e', 'EX_cyan_e', 'DM_aacald_c', 'DM_amob_c', 'DM_mththf_c', 'EX_glyb_e', 'EX_gthox_e', 'BIOMASS_Ec_iJO1366_WT_53p95M', 'EX_14glucan_e', 'EX_h2o2_e', 'EX_hacolipa_e', 'EX_halipa_e', 'EX_34dhpac_e', 'EX_hg2_e', 'EX_4hoxpacd_e', 'EX_cys__D_e', 'EX_acgal_e', 'EX_inost_e', 'EX_acolipa_e', 'EX_adocbl_e', 'EX_ag_e', 'EX_lipoate_e', 'EX_ala_B_e', 'EX_malthx_e', 'EX_maltpt_e', 'EX_malttr_e', 'EX_maltttr_e', 'EX_arbt_e', 'EX_meoh_e', 'EX_met__D_e', 'EX_aso3_e', 'EX_mg2_e', 'EX_mincyc_e', 'EX_btn_e', 'EX_mmet_e', 'EX_mn2_e', 'EX_ca2_e', 'EX_cbi_e', 'EX_mobd_e', 'EX_n2o_e', 'EX_na1_e', 'EX_nac_e', 'EX_cbl1_e', 'EX_cd2_e', 'EX_chol_e', 'EX_cl_e', 'EX_ni2_e', 'EX_nmn_e', 'EX_dms_e', 'EX_novbcn_e', 'EX_o16a4colipa_e', 'EX_doxrbcn_e', 'EX_o2s_e', 'EX_xtsn_e', 'EX_pacald_e', 'EX_zn2_e', '12DGR120tipp', '12DGR140tipp', '12DGR141tipp', '12DGR160tipp', 'EX_pnto__R_e', '12DGR161tipp', '12DGR18

In [3]:
i= 0
r = a.reactions[0]
l2= list()
l3= list()

for r in a.reactions:            #separates reactions that have biomass values for positive and negative flux values
    
    r = a.reactions[i]
    r_id = r.id
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))

    i3= int(((len(csvFile)-1)/2)+1)

    while i3< len(csvFile):
        inf= csvFile.iloc[i3,  2]
        flt= float(csvFile.iloc[i3,  2])
    
        if csvFile.iloc[i3,  2] == flt and r_id not in l2:
            l2.append(r_id)     #l2 contains reactions that have biomass values for positive flux values
        i3= i3+1
    
    i4= 0
    
    while i4< int(((len(csvFile)-1)/2)):
        inf= csvFile.iloc[i4,  2]
        flt= float(csvFile.iloc[i4,  2])
    
        if csvFile.iloc[i4,  2] == flt and r_id not in l3:
            l3.append(r_id)    #l3 contains reactions that have biomass values for negative flux values
        i4= i4+1

    i= i+1

D1= pd.DataFrame (l2, columns = ['Reaction names'])
D2= pd.DataFrame (l3, columns = ['Reaction names'])
b= D1.iloc[:, 0]
c= D2.iloc[:, 0]

union = pd.Series(np.union1d(b, c))
intersect = pd.Series(np.intersect1d(b, c))
bigb2= union[~union.isin(intersect)]

D3= pd.DataFrame (bigb2, columns = ['Reaction names'])
D3
D3.to_csv("res2.csv")      #contains reactions that have biomass values for either positive or negative flux values


In [4]:
i= 0
indecr_and_stable_above_zero= list()
diff_indecr_above_zero= list()
flat_above_zero= list()
one_to_three_data_point_above_zero= list()
r_id = a.reactions[0]


for r_id in a.reactions:    #separates everything that has an increasing/decreasing pattern from everything that has a saturating pattern and everything that is flat above zero
    l= list()
    n=0
    n2=0
    count=0
    r = a.reactions[i]
    r_id = r.id
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))

    while n < len(csvFile):
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l.append(csvFile.iloc[n,  2])
        n= n+1

    d= pd.DataFrame (l, columns = ['Biomass'])
    d.Biomass = d.Biomass.round(2)
    
    
    my_series = d['Biomass'].squeeze()


    if len(d)> 30 and r_id in l2 and r_id not in l3:
        
        while n2 < len(d):

            if d.iloc[n2,  0] > max(d["Biomass"])-0.01 and d.iloc[n2,  0] < max(d["Biomass"])+0.01:
                count= count+1
    
            n2= n2+1
        
        if count> 0.05*len(d) and count< 0.95*len(d):
            indecr_and_stable_above_zero.append(r_id)

        elif count> 0.95*len(d):
            flat_above_zero.append(r_id)
        
        else:
            diff_indecr_above_zero.append(r_id)

    
    if len(d)> 1 and len(d)<30  and r_id in l2 and r_id not in l3:
        
        while n2 < len(d):

            if d.iloc[n2,  0] > max(d["Biomass"])-0.01 and d.iloc[n2,  0] < max(d["Biomass"])+0.01:
                count= count+1
    
            n2= n2+1
        
        if count> 2 and count< len(d)-3:
            indecr_and_stable_above_zero.append(r_id)

        elif count> 1 and len(d) < 3:
            flat_above_zero.append(r_id)
        
        elif count> 2 and len(d) < 6:
            flat_above_zero.append(r_id)
        
        elif count> len(d)-3 and len(d)> 5:
            flat_above_zero.append(r_id)
        
        elif len(d)<4:
            one_to_three_data_point_above_zero.append(r_id)
        
        else:
            diff_indecr_above_zero.append(r_id)
    
    i= i+1

flat_above_zero
diff_indecr_above_zero
indecr_and_stable_above_zero


['2MAHMP',
 'AACPS8',
 'AACPS9',
 'ABTA',
 'ACCOAC',
 'ACONTa',
 'ACONTb',
 'ACPPAT120',
 'ACPPAT140',
 'ACPPAT141',
 'ACPPAT160',
 'ACPPAT161',
 'ACPPAT180',
 'ACPPAT181',
 'ALAALAr',
 'ALAR',
 'ADNCYC',
 'AP4AH',
 'AP4AS',
 'CMtex',
 'CD2tpp',
 'COBALT2tpp',
 'CS',
 'DASYN120',
 'DASYN140',
 'DASYN141',
 'DASYN160',
 'CU2abcpp',
 'DASYN161',
 'DASYN180',
 'DASYN181',
 'CU2tpp',
 'DOXRBCNtex',
 'ARGORNt7pp',
 'DTMPK',
 'EDA',
 'EDD',
 'DURADx',
 'ARGt3pp',
 'FACOAE60',
 'FBA3',
 'FHL',
 'FORtppi',
 'FUSAtex',
 'GK1',
 'G5SD',
 'G6PDH2r',
 'GLCDpp',
 'GLCNt2rpp',
 'GALUi',
 'GLNS',
 'GLU5K',
 'GF6PTA',
 'HYPOE',
 'HCO3E',
 'ICDHyr',
 'HEX4',
 'LCTStpp',
 'LYSt3pp',
 'MCOATA',
 'MALt3pp',
 'MAN1PT2',
 'MINCYCtex',
 'MELIBt2pp',
 'MELIBt3ipp',
 'MN2t3pp',
 'NADDP',
 'NADK',
 'NADPPPS',
 'GSPMDA',
 'GSPMDS',
 'NDPK2',
 'NDPK4',
 'NOVBCNtex',
 'NH4tpp',
 'NTD4',
 'NTD5',
 'GTPDPK',
 'O2tex',
 'P5CR',
 'GUACYC',
 'PGCD',
 'PDE1',
 'PDE4',
 'PDXPP',
 'PSERT',
 'PSP_L',
 'RBK',
 'RFAMPtex',
 

In [5]:
len(flat_above_zero)+len(diff_indecr_above_zero)+len(indecr_and_stable_above_zero)+len(one_to_three_data_point_above_zero)


639

In [6]:
i= 0
indecr_and_stable_below_zero= list()
diff_indecr_below_zero= list()
flat_below_zero= list()
one_to_three_data_point_below_zero= list()
r_id = a.reactions[0]


for r_id in a.reactions:    #separates everything that has an increasing/decreasing pattern from everything that has a saturating pattern and everything that is flat below zero
    l= list()
    n=0
    n2=0
    count=0
    r = a.reactions[i]
    r_id = r.id
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))

    while n < len(csvFile):
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l.append(csvFile.iloc[n,  2])
        n= n+1

    d= pd.DataFrame (l, columns = ['Biomass'])
    d.Biomass = d.Biomass.round(2)
    
    my_series = d['Biomass'].squeeze()
    

    if len(d)> 30 and r_id in l3 and r_id not in l2:
        
        while n2 < len(d):

            if d.iloc[n2,  0] > max(d["Biomass"])-0.01 and d.iloc[n2,  0] < max(d["Biomass"])+0.01:
                count= count+1
    
            n2= n2+1
        
        if count> 0.05*len(d) and count< 0.95*len(d):
            indecr_and_stable_below_zero.append(r_id)

        elif count> 0.95*len(d):
            flat_below_zero.append(r_id)
        
        else:
            diff_indecr_below_zero.append(r_id)

    
    if len(d)> 1 and len(d)<30  and r_id in l3 and r_id not in l2:
        
        while n2 < len(d):

            if d.iloc[n2,  0] > max(d["Biomass"])-0.01 and d.iloc[n2,  0] < max(d["Biomass"])+0.01:
                count= count+1
    
            n2= n2+1
        
        if count> 2 and count< len(d)-3:
            indecr_and_stable_below_zero.append(r_id)

        elif count> 1 and len(d) < 3:
            flat_below_zero.append(r_id)
        
        elif count> 2 and len(d) < 6:
            flat_below_zero.append(r_id)
        
        elif count> len(d)-3 and len(d)> 5:
            flat_below_zero.append(r_id)
        
        elif len(d)<4:
            one_to_three_data_point_below_zero.append(r_id)
        
        else:
            diff_indecr_below_zero.append(r_id)
    
    i= i+1

flat_below_zero
diff_indecr_below_zero
indecr_and_stable_below_zero


['EX_csn_e',
 'EX_cynt_e',
 'EX_no_e',
 'EX_dmso_e',
 'EX_no3_e',
 'EX_o2_e',
 'EX_tmao_e',
 '2DGULRGx',
 '2DGULRGy',
 '2DGULRx',
 'ADK4',
 'ASPTA',
 'ATHRDHr',
 'CHLt2pp',
 'BETALDHx',
 'BETALDHy',
 'BSORx',
 'BSORy',
 'CTBTt2rpp',
 'CUt3',
 'DKGLCNR2y',
 'DOGULNR',
 'ASNt2rpp',
 'FRUpts2pp',
 'FUMt2_2pp',
 'FUMt2_3pp',
 'G2PPpp',
 'G3PD2',
 'ASPO6',
 'GLNabcpp',
 'GALt2pp',
 'GLUNpp',
 'GLYBt2pp',
 'GUAtpp',
 'HEXt2rpp',
 'HG2t3pp',
 'ILEt2rpp',
 'HISt2rpp',
 'LACZ',
 'LEUt2rpp',
 'LIPOCT',
 'MLTP1',
 'MLTP2',
 'MLTP3',
 'MG2t3_2pp',
 'NODOx',
 'NODOy',
 'NO3R1bpp',
 'NO3R2bpp',
 'OMMBLHX',
 'OMMBLHX3',
 'OMPHHX',
 'OMPHHX3',
 'OPHHX',
 'OPHHX3',
 'PGMT',
 'PTRCORNt7pp',
 'SUCFUMtpp',
 'SUCTARTtpp',
 'TARTD',
 'TREptspp',
 'PMANM',
 'XANtpp',
 'VALt2rpp',
 'MECDPDH2',
 'XYLt2pp',
 '2DGLCNRx',
 '2DGLCNRy',
 'DOGULND',
 'TARTDC']

In [7]:
i2= 0
mon_incr_above_zero= list()
mon_decr_above_zero= list()

if len(diff_indecr_above_zero)> 0:
    
    for r_id in diff_indecr_above_zero:                    #this code separates reactions with an increasing and stable trend from reactions with a decreasing and stable trend above zero
    
        i=0
        r_id = diff_indecr_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        l= list()

        while i < len(csvFile):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)

        my_series = csvFile2['Biomass'].squeeze()
        type(my_series)

        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                mon_decr_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                mon_incr_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(diff_indecr_above_zero)
set2= set(mon_decr_above_zero)
set3= set(mon_incr_above_zero)
inANDdecr_above_zero= list(set1-set2-set3)
inANDdecr_above_zero


['ADSL2r',
 'AGPAT161',
 'A5PISO',
 'G1SAT',
 'PRAGSr',
 'OMCDC',
 'ACGK',
 'METAT',
 'PSSA161',
 'ANPRT',
 'DHQS',
 'DM_5drib_c',
 'IPMD',
 'NADS1',
 'DHQTi',
 'ACLS',
 'UAGCVT',
 'HSST',
 'ARGSS',
 'UHGADA',
 'GLUTRR',
 'DDPA',
 'LEUTAi',
 '5DOAN',
 'TMDS',
 'K2L4Aabctex',
 'PRASCSi',
 'AIRC2',
 'O2tpp',
 'SERAT',
 'UAMAGS',
 'UAGPT3',
 'FCLT',
 'THRS',
 'ADSS',
 'DHAD1',
 'APG3PAT161',
 'PRAIi',
 'AGPAT160',
 'PRAIS',
 'GMPS2',
 'U23GAAT',
 'PRAMPC',
 'HSK',
 'THRD_L',
 'CYSTL',
 'UDCPDP',
 'NNATr',
 'PItex',
 'AICART',
 'KDOPP',
 'SHKK',
 'DHDPS',
 'ACHBS',
 'UPPDC1',
 'PPNDH',
 'DHFR',
 'ASPCT',
 'G1PACT',
 'ARGSL',
 'PSCVT',
 'PRATPP',
 'APG3PAT160',
 'HSTPT',
 'UPP3S',
 'G5SADs',
 'MTHFR2',
 '3OAR140',
 'ADSK',
 'UAGDP',
 'SADT2',
 'PE161abcpp',
 'PPBNGS',
 'PSD160',
 'IPPS',
 'PAPPT3',
 'HMBS',
 'K2L4Aabcpp',
 'UAMAS',
 'IGPDH',
 'OMPDC',
 'HISTP',
 'KDOPS',
 'USHD',
 'DHAD2',
 'SHSL1',
 'ATPPRT',
 'ANS',
 'PE160abcpp',
 'KDOCT2',
 'UGMDDS',
 'MOAT',
 'IG3PS',
 'SO4tex',
 'KARA

In [8]:
w= list()
w2= list()
i2=0

if len(inANDdecr_above_zero)> 0:
    
    while i2 < len(inANDdecr_above_zero):        #this code finds reactions that are not truly both increasing and decreasing and stable above zero
    
        i=0
        r_id = inANDdecr_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        l= list()

        while i < len(csvFile):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2] < max(l) and (l[n2+3] < max(l) or l[n2+3] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w.append(r_id)

    
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and (l[n2-3]<max(l) or l[n2-3] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w2.append(r_id)

        i2= i2+1

w


['AGPAT161',
 'A5PISO',
 'G1SAT',
 'METAT',
 'PSSA161',
 'ANPRT',
 'DM_5drib_c',
 'NADS1',
 'UAGCVT',
 'HSST',
 'UHGADA',
 'GLUTRR',
 '5DOAN',
 'TMDS',
 'K2L4Aabctex',
 'UAMAGS',
 'UAGPT3',
 'FCLT',
 'APG3PAT161',
 'PRAIi',
 'AGPAT160',
 'U23GAAT',
 'PRAMPC',
 'CYSTL',
 'UDCPDP',
 'NNATr',
 'KDOPP',
 'UPPDC1',
 'PPNDH',
 'DHFR',
 'G1PACT',
 'PRATPP',
 'APG3PAT160',
 'HSTPT',
 'UPP3S',
 'MTHFR2',
 '3OAR140',
 'UAGDP',
 'PE161abcpp',
 'PPBNGS',
 'PSD160',
 'PAPPT3',
 'HMBS',
 'K2L4Aabcpp',
 'UAMAS',
 'IGPDH',
 'HISTP',
 'KDOPS',
 'USHD',
 'SHSL1',
 'ATPPRT',
 'ANS',
 'PE160abcpp',
 'KDOCT2',
 'UGMDDS',
 'MOAT',
 'IG3PS',
 'METS',
 'PSD161',
 'MCTP1App',
 'PSSA160',
 'CTPS2',
 'LPADSS',
 'PPND',
 'UAGAAT',
 'UAAGDS',
 'IGPS',
 'UAPGR',
 'TDSK',
 'GLUTRS',
 'HISTD',
 '3OAS140',
 'PRMICI',
 'MPTG',
 'MOAT2']

In [9]:
union = pd.Series(np.union1d(w, w2))
unclassified_above_zero = pd.Series(np.intersect1d(w, w2))

set1= set(w)
set2= set(unclassified_above_zero)
set3= set1-set2
w= list(set3)

set1= set(w2)
set2= set(unclassified_above_zero)
set3= set1-set2
w2= list(set3)

unclassified_above_zero= list(unclassified_above_zero)
unclassified_above_zero


[]

In [10]:
set1= set(inANDdecr_above_zero)
set2= set(w)
set3= set(unclassified_above_zero)
set4= set1-set2-set3
inANDdecr_above_zero= list(set4)

mon_decr_above_zero= list(mon_decr_above_zero) + list(w)


set1= set(inANDdecr_above_zero)
set2= set(w2)
set3= set1-set2
inANDdecr_above_zero= list(set3)

mon_incr_above_zero= list(mon_incr_above_zero) + list(w2)


In [11]:
i2= 0
mon_incr_below_zero= list()
mon_decr_below_zero= list()

if len(diff_indecr_below_zero)> 0:
    
    for r_id in diff_indecr_below_zero:                    #this code separates reactions with an increasing and stable trend from reactions with a decreasing and stable trend above zero
    
        i=0
        r_id = diff_indecr_below_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        l= list()

        while i < len(csvFile):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)

        my_series = csvFile2['Biomass'].squeeze()
        type(my_series)

        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                mon_decr_below_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                mon_incr_below_zero.append(r_id)

    
        i2= i2 + 1

set1= set(diff_indecr_below_zero)
set2= set(mon_decr_below_zero)
set3= set(mon_incr_below_zero)
inANDdecr_below_zero= list(set1-set2-set3)
inANDdecr_below_zero


['SDPTA',
 'EX_pi_e',
 'ASAD',
 'ILETA',
 'AGPR',
 'EX_metsox_R__L_e',
 'ACOTA',
 'ORPT',
 'IPPMIa',
 'PGAMT',
 'IPPMIb',
 'KARA1',
 'EX_metsox_S__L_e',
 'EX_sulfac_e',
 'EX_nh4_e',
 'TYRTA',
 'HSDy',
 'GLUR',
 'EX_orot_e',
 'IMPC',
 'PHETA1',
 'EX_26dap__M_e',
 'EX_isetac_e',
 'DHORTS',
 'EX_met__L_e',
 'EX_so4_e',
 'AIRC3',
 'EX_skm_e',
 'EX_mso3_e']

In [12]:
w= list()
w2= list()
i2=0

if len(inANDdecr_below_zero)> 0:
    
    while i2 < len(inANDdecr_below_zero):        #this code finds reactions that are not truly both increasing and decreasing and stable above zero
    
        i=0
        r_id = inANDdecr_below_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        l= list()

        while i < len(csvFile):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and (l[n2+3] < max(l) or l[n2+3] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w.append(r_id)

    
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and (l[n2-3] < max(l) or l[n2-3] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w2.append(r_id)

    
        i2= i2+1

w


[]

In [13]:
union = pd.Series(np.union1d(w, w2))
unclassified_below_zero = pd.Series(np.intersect1d(w, w2))

set1= set(w)
set2= set(unclassified_below_zero)
set3= set1-set2
w= list(set3)

set1= set(w2)
set2= set(unclassified_below_zero)
set3= set1-set2
w2= list(set3)

unclassified_below_zero= list(unclassified_below_zero)
unclassified_below_zero


[]

In [14]:
set1= set(inANDdecr_below_zero)
set2= set(w)
set3= set(unclassified_below_zero)
set4= set1-set2-set3
inANDdecr_below_zero= list(set4)

mon_decr_below_zero= list(mon_decr_below_zero) + list(w)


set1= set(inANDdecr_below_zero)
set2= set(w2)
set3= set1-set2
inANDdecr_below_zero= list(set3)

mon_incr_below_zero= list(mon_incr_below_zero) + list(w2)


In [15]:
i2= 0
mon_incr_and_stable_above_zero= list()
mon_decr_and_stable_above_zero= list()

if len(indecr_and_stable_above_zero)> 0:
    
    for r_id in indecr_and_stable_above_zero:                    #this code separates reactions with an increasing and stable trend from reactions with a decreasing and stable trend above zero
    
        i=0
        r_id = indecr_and_stable_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        l= list()

        while i < len(csvFile):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)

        my_series = csvFile2['Biomass'].squeeze()
        type(my_series)

        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                mon_decr_and_stable_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                mon_incr_and_stable_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(indecr_and_stable_above_zero)
set2= set(mon_decr_and_stable_above_zero)
set3= set(mon_incr_and_stable_above_zero)
inANDdecr_and_stable_above_zero= list(set1-set2-set3)
inANDdecr_and_stable_above_zero


['ACONTa',
 'ACPPAT161',
 'DASYN160',
 'ACPPAT160',
 'MCOATA',
 'DTMPK',
 'GK1',
 'PSERT',
 'ACCOAC',
 'P5CR',
 'PSP_L',
 'PMPK',
 'DASYN161',
 'ACONTb',
 'GLNS',
 'PGCD',
 'GF6PTA',
 'CU2tpp',
 'UMPK',
 'NDPK2',
 'NDPK4',
 'ALAALAr',
 'TMPK',
 'NADK',
 'HCO3E',
 'ICDHyr',
 'CS',
 'COBALT2tpp',
 'ALAR',
 'NH4tpp']

In [16]:
w= list()
w2= list()
i2=0

if len(inANDdecr_and_stable_above_zero)> 0:
    
    while i2 < len(inANDdecr_and_stable_above_zero):        #this code finds reactions that are not truly both increasing and decreasing and stable above zero
    
        i=0
        r_id = inANDdecr_and_stable_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        l= list()

        while i < len(csvFile):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and (l[n2+3] < max(l) or l[n2+3] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w.append(r_id)

    
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and (l[n2-3] < max(l) or l[n2-3] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w2.append(r_id)

    
        i2= i2+1

w


['ACPPAT161',
 'DASYN160',
 'ACPPAT160',
 'MCOATA',
 'DTMPK',
 'ACCOAC',
 'PMPK',
 'DASYN161',
 'GF6PTA',
 'CU2tpp',
 'NDPK4',
 'ALAALAr',
 'TMPK',
 'NADK',
 'COBALT2tpp',
 'ALAR']

In [17]:
union = pd.Series(np.union1d(w, w2))
unclassified_above_zero2 = pd.Series(np.intersect1d(w, w2))

set1= set(w)
set2= set(unclassified_above_zero2)
set3= set1-set2
w= list(set3)

set1= set(w2)
set2= set(unclassified_above_zero2)
set3= set1-set2
w2= list(set3)

unclassified_above_zero= list(unclassified_above_zero2)+list(unclassified_above_zero)
unclassified_above_zero


[]

In [18]:
set1= set(inANDdecr_and_stable_above_zero)
set2= set(w)
set3= set(unclassified_above_zero2)
set4= set1-set2-set3
inANDdecr_and_stable_above_zero= list(set4)

mon_decr_and_stable_above_zero= list(mon_decr_and_stable_above_zero) + list(w)


set1= set(inANDdecr_and_stable_above_zero)
set2= set(w2)
set3= set1-set2
inANDdecr_and_stable_above_zero= list(set3)

mon_incr_and_stable_above_zero= list(mon_incr_and_stable_above_zero) + list(w2)


In [19]:
i2= 0
mon_incr_and_stable_below_zero= list()
mon_decr_and_stable_below_zero= list()

if len(indecr_and_stable_below_zero)> 0:
    
    for r_id in indecr_and_stable_below_zero:                    #this code separates reactions with an increasing and stable trend from reactions with a decreasing and stable trend below zero
    
        i=0
        r_id = indecr_and_stable_below_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        l= list()

        while i < len(csvFile):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)

        my_series = csvFile2['Biomass'].squeeze()
        type(my_series)

        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                mon_decr_and_stable_below_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                mon_incr_and_stable_below_zero.append(r_id)

        i2= i2 + 1

set1= set(indecr_and_stable_below_zero)
set2= set(mon_decr_and_stable_below_zero)
set3= set(mon_incr_and_stable_below_zero)
inANDdecr_and_stable_below_zero= list(set1-set2-set3)
inANDdecr_and_stable_below_zero


['ASPTA', 'G3PD2']

In [20]:
w3= list()
w4= list()
i2=0

if len(inANDdecr_and_stable_below_zero)>0:
    
    while i2 < len(inANDdecr_and_stable_below_zero):       #this code finds reactions that are not truly both increasing and decreasing and stable below zero
    
        i=0
        r_id = inANDdecr_and_stable_below_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        l= list()

        while i < len(csvFile):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and (l[n2+3] < max(l) or l[n2+3] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w.append(r_id)

    
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and (l[n2-3] < max(l) or l[n2-3] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w2.append(r_id)

    
        i2= i2+1

w3


[]

In [21]:
union = pd.Series(np.union1d(w, w2))
unclassified_below_zero2 = pd.Series(np.intersect1d(w, w2))

set1= set(w)
set2= set(unclassified_below_zero2)
set3= set1-set2
w= list(set3)

set1= set(w2)
set2= set(unclassified_below_zero2)
set3= set1-set2
w2= list(set3)

unclassified_below_zero= list(unclassified_below_zero2)+list(unclassified_below_zero)
unclassified_below_zero


[]

In [22]:
set1= set(inANDdecr_and_stable_below_zero)
set2= set(w3)
set3= set(w4)
set4= set(unclassified_below_zero2)
set5= set1-set2-set3-set4
inANDdecr_and_stable_below_zero= list(set5)

mon_decr_and_stable_below_zero= list(mon_decr_and_stable_below_zero) + list(w3)
mon_incr_and_stable_below_zero= list(mon_incr_and_stable_below_zero) + list(w4)


In [23]:
i= 0
over_below_zero_and_under_above_zero= list()
under_below_zero_and_over_above_zero= list()
over_below_zero_and_over_above_zero= list()
under_below_zero_and_under_above_zero= list()
r_id = a.reactions[0]

for r_id in a.reactions:    #separates reactions based on whether they have less than 30 or not values both in the above and the below zero interval
    l= list()
    n=0
    n2=0
    count=0
    count2=0
    r = a.reactions[i]
    r_id = r.id
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))

    while n < int(((len(csvFile)-1)/2)+1):
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l.append(csvFile.iloc[n,  2])
        n= n+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    
    
    n= int((len(csvFile)-1)/2)
    l4= list()
    
    while n < len(csvFile):
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l4.append(csvFile.iloc[n,  2])
        n= n+1

    csvFile3= pd.DataFrame (l4, columns = ['Biomass'])
    csvFile3.Biomass = csvFile3.Biomass.round(2)
    
    
    my_series = csvFile2['Biomass'].squeeze()
    my_series2 = csvFile3['Biomass'].squeeze()


    if (len(csvFile2)>30 or len(csvFile2)==30) and len(csvFile3)< 30 and r_id in l3 and r_id in l2:
        
        over_below_zero_and_under_above_zero.append(r_id)
        
    if len(csvFile2)<30 and (len(csvFile3)> 30 or len(csvFile3)==30) and r_id in l3 and r_id in l2:
    
        under_below_zero_and_over_above_zero.append(r_id)
    
    if (len(csvFile2)>30 or len(csvFile2)==30) and (len(csvFile3)> 30 or len(csvFile3)==30) and r_id in l3 and r_id in l2:
    
        over_below_zero_and_over_above_zero.append(r_id)
        
    if len(csvFile2)<30 and len(csvFile3)< 30 and r_id in l3 and r_id in l2:
    
        under_below_zero_and_under_above_zero.append(r_id)
    
    
    i= i+1
    
over_below_zero_and_under_above_zero


['EX_LalaDgluMdapDala_e',
 'EX_anhgm_e',
 'EX_enter_e',
 'EX_feenter_e',
 'CLIPAabctex',
 'CPPPGO',
 'ECOAH3',
 'ECOAH4',
 'ECOAH5',
 'ECOAH6',
 'ECOAH7',
 'LIPAabctex',
 'RPI',
 'PPPGO',
 'PPPGO3']

In [24]:
i= 0
flat_below_zero_and_indecr_and_stable_above_zero= list()
diff_indecr_below_zero_and_diff_indecr_above_zero= list()
rest_both_positive_and_negative= list()
indecr_and_stable_below_zero_and_indecr_and_stable_above_zero= list()
indecr_and_stable_below_zero_and_flat_above_zero= list()
flat_below_zero_and_flat_above_zero= list()
diff_indecr_below_zero_and_flat_above_zero= list()
flat_below_zero_and_diff_indecr_above_zero= list()
indecr_and_stable_below_zero_and_flat_above_zero= list()
indecr_and_stable_below_zero_and_diff_indecr_above_zero= list()
flat_below_zero_and_indecr_and_stable_above_zero= list()
diff_indecr_below_zero_and_indecr_and_stable_above_zero= list()
indecr_and_stable_below_zero_and_diff_indecr_above_zero= list()
indecr_and_stable_below_zero_and_one_to_three_data_point_above_zero= list()
flat_below_zero_and_one_to_three_data_point_above_zero= list()
diff_indecr_below_zero_and_one_to_three_data_point_above_zero= list()

if len(over_below_zero_and_under_above_zero)> 0:

    for r_id in over_below_zero_and_under_above_zero:    #separates over_below_zero_and_under_above_zero reactions in more specific subcategories
    
        l= list()
        n=0
        n2=0
        count=0
        count2=0
        r_id = over_below_zero_and_under_above_zero[i]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))

        while n < int(((len(csvFile)-1)/2)+1):
            inf= csvFile.iloc[n,  2]
            flt= float(csvFile.iloc[n,  2])
    
            if csvFile.iloc[n,  2] == flt:
                l.append(csvFile.iloc[n,  2])
            n= n+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
    
    
        n= int((len(csvFile)-1)/2)
        l4= list()
    
        while n < len(csvFile):
            inf= csvFile.iloc[n,  2]
            flt= float(csvFile.iloc[n,  2])
    
            if csvFile.iloc[n,  2] == flt:
                l4.append(csvFile.iloc[n,  2])
            n= n+1

        csvFile3= pd.DataFrame (l4, columns = ['Biomass'])
        csvFile3.Biomass = csvFile3.Biomass.round(2)
    
    
        my_series = csvFile2['Biomass'].squeeze()
        my_series2 = csvFile3['Biomass'].squeeze()


        while n2 < len(csvFile2):

            if csvFile2.iloc[n2,  0] > max(csvFile2["Biomass"])-0.01 and csvFile2.iloc[n2,  0] < max(csvFile2["Biomass"])+0.01:
                count= count+1
    
            n2= n2+1
        
        n2=0
        while n2 < len(csvFile3):

            if csvFile3.iloc[n2,  0] > max(csvFile3["Biomass"])-0.01 and csvFile3.iloc[n2,  0] < max(csvFile3["Biomass"])+0.01:
                count2= count2+1
    
            n2= n2+1
    
        if count== 0.05*len(csvFile2) or count> 0.05*len(csvFile2) and count< 0.95*len(csvFile2) and len(csvFile3)<4:
            indecr_and_stable_below_zero_and_one_to_three_data_point_above_zero.append(r_id)
    
        elif count== 0.05*len(csvFile2) or count> 0.05*len(csvFile2) and count< 0.95*len(csvFile2) and (count2> 2 and count2< len(csvFile3)-3):        
            indecr_and_stable_below_zero_and_indecr_and_stable_above_zero.append(r_id)
        
        elif (count> 0.05*len(csvFile2) and count< 0.95*len(csvFile2)) and (count2> len(csvFile3)-3 and len(csvFile3)>5):
            indecr_and_stable_below_zero_and_flat_above_zero.append(r_id)
    
        elif (count> 0.05*len(csvFile2) and count< 0.95*len(csvFile2)) and (count2> 2 and len(csvFile3) < 6):
            indecr_and_stable_below_zero_and_flat_above_zero.append(r_id)
    
        elif (count> 0.05*len(csvFile2) and count< 0.95*len(csvFile2)) and (count2> 1 and len(csvFile3) < 3):
            indecr_and_stable_below_zero_and_flat_above_zero.append(r_id)
    
        elif (count == 0.05*len(csvFile2) or count> 0.05*len(csvFile2) and count< 0.95*len(csvFile2)) and count2< 3:
            indecr_and_stable_below_zero_and_diff_indecr_above_zero.append(r_id)
    
    
    
        elif (count == 0.95*len(csvFile2) or count> 0.95*len(csvFile2)) and len(csvFile3)<4:
            flat_below_zero_and_one_to_three_data_point_above_zero.append(r_id)
    
        elif (count == 0.95*len(csvFile2) or count> 0.95*len(csvFile2)) and (count2> 2 and count2< len(csvFile3)-3):
            flat_below_zero_and_indecr_and_stable_above_zero.append(r_id)
    
        elif (count == 0.95*len(csvFile2) or count> 0.95*len(csvFile2)) and count2< 3:
            flat_below_zero_and_diff_indecr_above_zero.append(r_id)
   
        elif (count == 0.95*len(csvFile2) or count> 0.95*len(csvFile2)) and (count2> len(csvFile3)-3 and len(csvFile3)>5):
            flat_below_zero_and_flat_above_zero.append(r_id)
    
        elif (count == 0.95*len(csvFile2) or count> 0.95*len(csvFile2)) and (count2> 2 and len(csvFile3) < 6):
            flat_below_zero_and_flat_above_zero.append(r_id)
        
        elif (count == 0.95*len(csvFile2) or count> 0.95*len(csvFile2)) and (count2> 1 and len(csvFile3) < 3):
            flat_below_zero_and_flat_above_zero.append(r_id)

    
        elif count< 0.05*len(csvFile2) and len(csvFile3)<4:
            diff_indecr_below_zero_and_one_to_three_data_point_above_zero.append(r_id)
    
        elif count< 0.05*len(csvFile2) and (count2> 2 and count2< len(csvFile3)-3):
            diff_indecr_below_zero_and_indecr_and_stable_above_zero.append(r_id)
        
        elif count< 0.05*len(csvFile2) and count2< 3:
            diff_indecr_below_zero_and_diff_indecr_above_zero.append(r_id)

        elif count< 0.05*len(csvFile2) and (count2> len(csvFile3)-3 and len(csvFile3)>5):
            diff_indecr_below_zero_and_flat_above_zero.append(r_id)
        
        elif count< 0.05*len(csvFile2) and (count2> 2 and len(csvFile3) < 6):
            diff_indecr_below_zero_and_flat_above_zero.append(r_id)
    
        elif count< 0.05*len(csvFile2) and (count2> 1 and len(csvFile3) < 3):
            diff_indecr_below_zero_and_flat_above_zero.append(r_id)
    
    
        i= i+1

indecr_and_stable_below_zero_and_indecr_and_stable_above_zero


[]

In [25]:
i=0
one_to_three_data_point_below_zero_and_diff_indecr_above_zero= list()
one_to_three_data_point_below_zero_and_indecr_and_stable_above_zero= list()
one_to_three_data_point_below_zero_and_flat_above_zero= list()

if len(under_below_zero_and_over_above_zero)> 0:

    for r_id in under_below_zero_and_over_above_zero:    #separates under_below_zero_and_over_above_zero reactions in more specific subcategories
        l= list()
        n=0
        n2=0
        count=0
        count2=0
        r_id = under_below_zero_and_over_above_zero[i]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))

        while n < int(((len(csvFile)-1)/2)+1):
            inf= csvFile.iloc[n,  2]
            flt= float(csvFile.iloc[n,  2])
    
            if csvFile.iloc[n,  2] == flt:
                l.append(csvFile.iloc[n,  2])
            n= n+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
    
    
        n= int((len(csvFile)-1)/2)
        l4= list()
    
        while n < len(csvFile):
            inf= csvFile.iloc[n,  2]
            flt= float(csvFile.iloc[n,  2])
    
            if csvFile.iloc[n,  2] == flt:
                l4.append(csvFile.iloc[n,  2])
            n= n+1

        csvFile3= pd.DataFrame (l4, columns = ['Biomass'])
        csvFile3.Biomass = csvFile3.Biomass.round(2)
    
    
        my_series = csvFile2['Biomass'].squeeze()
        my_series2 = csvFile3['Biomass'].squeeze()


        while n2 < len(csvFile2):

            if csvFile2.iloc[n2,  0] > max(csvFile2["Biomass"])-0.01 and csvFile2.iloc[n2,  0] < max(csvFile2["Biomass"])+0.01:
                count= count+1
    
            n2= n2+1
        
        n2=0
        while n2 < len(csvFile3):

            if csvFile3.iloc[n2,  0] > max(csvFile3["Biomass"])-0.01 and csvFile3.iloc[n2,  0] < max(csvFile3["Biomass"])+0.01:
                count2= count2+1
    
            n2= n2+1
    
        if len(csvFile2)<4 and (count2== 0.05*len(csvFile3) or count2> 0.05*len(csvFile3) and count2< 0.95*len(csvFile3)):
            one_to_three_data_point_below_zero_and_indecr_and_stable_above_zero.append(r_id)
    
        elif (count> 2 and count< len(csvFile2)-3) and (count2== 0.05*len(csvFile3) or count2> 0.05*len(csvFile3) and count2< 0.95*len(csvFile3)):        
            indecr_and_stable_below_zero_and_indecr_and_stable_above_zero.append(r_id)
        
        elif (count> len(csvFile2)-3 and len(csvFile2)>5) and (count2> 0.05*len(csvFile3) and count2< 0.95*len(csvFile3)):
            flat_below_zero_and_indecr_and_stable_above_zero.append(r_id)
    
        elif (count> 2 and len(csvFile2) < 6) and (count2> 0.05*len(csvFile3) and count2< 0.95*len(csvFile3)):
            flat_below_zero_and_indecr_and_stable_above_zero.append(r_id)
    
        elif (count> 1 and len(csvFile2) < 3) and (count2> 0.05*len(csvFile3) and count2< 0.95*len(csvFile3)):
            flat_below_zero_and_indecr_and_stable_above_zero.append(r_id)
    
        elif count< 3 and (count2 == 0.05*len(csvFile3) or count2> 0.05*len(csvFile3) and count2< 0.95*len(csvFile3)):
            diff_indecr_below_zero_and_indecr_and_stable_above_zero.append(r_id)
    
    
    
        elif len(csvFile2)<4 and (count2 == 0.95*len(csvFile3) or count2> 0.95*len(csvFile3)):
            one_to_three_data_point_below_zero_and_flat_above_zero.append(r_id)
    
        elif (count> 2 and count< len(csvFile2)-3) and (count2 == 0.95*len(csvFile3) or count2> 0.95*len(csvFile3)):
            indecr_and_stable_below_zero_and_flat_above_zero.append(r_id)
    
        elif count<3 and (count2 == 0.95*len(csvFile3) or count2> 0.95*len(csvFile3)):
            diff_indecr_below_zero_and_flat_above_zero.append(r_id)
   
        elif (count> len(csvFile2)-3 and len(csvFile2)>5) and (count2 == 0.95*len(csvFile3) or count2> 0.95*len(csvFile3)):
            flat_below_zero_and_flat_above_zero.append(r_id)
    
        elif (count> 2 and len(csvFile2) < 6) and (count2 == 0.95*len(csvFile3) or count2> 0.95*len(csvFile3)):
            flat_below_zero_and_flat_above_zero.append(r_id)
    
        elif (count> 1 and len(csvFile2) < 3) and (count2 == 0.95*len(csvFile3) or count2> 0.95*len(csvFile3)):
            flat_below_zero_and_flat_above_zero.append(r_id)

            
    
        elif len(csvFile2)<4 and count2< 0.05*len(csvFile3):
            one_to_three_data_point_below_zero_and_diff_indecr_above_zero.append(r_id)
    
        elif (count> 2 and count< len(csvFile2)-3) and count2< 0.05*len(csvFile3):
            indecr_and_stable_below_zero_and_diff_indecr_above_zero.append(r_id)
        
        elif count< 3 and count2< 0.05*len(csvFile3):
            diff_indecr_below_zero_and_diff_indecr_above_zero.append(r_id)

        elif (count> len(csvFile2)-3 and len(csvFile2)>5) and count2< 0.05*len(csvFile3):
            flat_below_zero_and_diff_indecr_above_zero.append(r_id)
    
        elif (count> 2 and len(csvFile2) < 6) and count2< 0.05*len(csvFile3):
            flat_below_zero_and_diff_indecr_above_zero.append(r_id)
    
        elif (count> 1 and len(csvFile2) < 3) and count2< 0.05*len(csvFile3):
            flat_below_zero_and_diff_indecr_above_zero.append(r_id)
        
    
        i= i+1

indecr_and_stable_below_zero_and_indecr_and_stable_above_zero


[]

In [26]:
i=0

if len(over_below_zero_and_over_above_zero)> 0:

    for r_id in over_below_zero_and_over_above_zero:    #separates over_below_zero_and_over_above_zero reactions in more specific subcategories
        l= list()
        n=0
        n2=0
        count=0
        count2=0
        r_id = over_below_zero_and_over_above_zero[i]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))

        while n < int(((len(csvFile)-1)/2)+1):
            inf= csvFile.iloc[n,  2]
            flt= float(csvFile.iloc[n,  2])
    
            if csvFile.iloc[n,  2] == flt:
                l.append(csvFile.iloc[n,  2])
            n= n+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
    
    
        n= int((len(csvFile)-1)/2)
        l4= list()
    
        while n < len(csvFile):
            inf= csvFile.iloc[n,  2]
            flt= float(csvFile.iloc[n,  2])
    
            if csvFile.iloc[n,  2] == flt:
                l4.append(csvFile.iloc[n,  2])
            n= n+1

        csvFile3= pd.DataFrame (l4, columns = ['Biomass'])
        csvFile3.Biomass = csvFile3.Biomass.round(2)
    
    
        my_series = csvFile2['Biomass'].squeeze()
        my_series2 = csvFile3['Biomass'].squeeze()


        while n2 < len(csvFile2):

            if csvFile2.iloc[n2,  0] > max(csvFile2["Biomass"])-0.01 and csvFile2.iloc[n2,  0] < max(csvFile2["Biomass"])+0.01:
                count= count+1
    
            n2= n2+1
        
        n2=0
        while n2 < len(csvFile3):

            if csvFile3.iloc[n2,  0] > max(csvFile3["Biomass"])-0.01 and csvFile3.iloc[n2,  0] < max(csvFile3["Biomass"])+0.01:
                count2= count2+1
    
            n2= n2+1
    
        if (count== 0.05*len(csvFile2) or count> 0.05*len(csvFile2) and count< 0.95*len(csvFile2)) and (count2== 0.05*len(csvFile3) or count2> 0.05*len(csvFile3) and count2< 0.95*len(csvFile3)):        
            indecr_and_stable_below_zero_and_indecr_and_stable_above_zero.append(r_id)
        
        elif (count == 0.95*len(csvFile2) or count> 0.95*len(csvFile2)) and (count2> 0.05*len(csvFile3) and count2< 0.95*len(csvFile3)):
            flat_below_zero_and_indecr_and_stable_above_zero.append(r_id)
    
        elif count< 0.05*len(csvFile2) and (count2 == 0.05*len(csvFile3) or count2> 0.05*len(csvFile3) and count2< 0.95*len(csvFile3)):
            diff_indecr_below_zero_and_indecr_and_stable_above_zero.append(r_id)
    
    
    
        elif (count== 0.05*len(csvFile2) or count> 0.05*len(csvFile2) and count<0.95*len(csvFile2)) and (count2 == 0.95*len(csvFile3) or count2> 0.95*len(csvFile3)):
            indecr_and_stable_below_zero_and_flat_above_zero.append(r_id)
    
        elif count< 0.05*len(csvFile2) and (count2 == 0.95*len(csvFile3) or count2> 0.95*len(csvFile3)):
            diff_indecr_below_zero_and_flat_above_zero.append(r_id)
   
        elif (count == 0.95*len(csvFile2) or count> 0.95*len(csvFile2)) and (count2 == 0.95*len(csvFile3) or count2> 0.95*len(csvFile3)):
            flat_below_zero_and_flat_above_zero.append(r_id)

    
    
        elif (count== 0.05*len(csvFile2) or count> 0.05*len(csvFile2) and count< 0.95*len(csvFile2)) and count2< 0.05*len(csvFile3):
            indecr_and_stable_below_zero_and_diff_indecr_above_zero.append(r_id)
        
        elif count< 0.05*len(csvFile2) and count2< 0.05*len(csvFile3):
            diff_indecr_below_zero_and_diff_indecr_above_zero.append(r_id)

        elif (count == 0.95*len(csvFile2) or count> 0.95*len(csvFile2)) and count2< 0.05*len(csvFile3):
            flat_below_zero_and_diff_indecr_above_zero.append(r_id)
    
    
        i= i+1

indecr_and_stable_below_zero_and_indecr_and_stable_above_zero


['ACALD',
 'ALDD2y',
 'ASPt2_2pp',
 'ASPt2_3pp',
 'CITL',
 'CMtpp',
 'CD2t3pp',
 'COBALT2t3pp',
 'CYStpp',
 'CYTBD2pp',
 'CYTBDpp',
 'CYTBO3_4pp',
 'ARBt2rpp',
 'DAAD',
 'ARBt3ipp',
 'DOXRBCNtpp',
 'F6PP',
 'FADRx',
 'FADRx2',
 'FBA',
 'FBP',
 'FE2t2pp',
 'ASNS2',
 'FLDR2',
 'FE2tpp',
 'FORt2pp',
 'FUSAtpp',
 'G6PP',
 'GLCptspp',
 'GLCt2pp',
 'ASPT',
 'GDPDPK',
 'GDPMNH',
 'GDPTPDP',
 'GUAt2pp',
 'GLYCLTDx',
 'GLYCLTDy',
 'IDOND2',
 'GLYK',
 'HPYRRy',
 'Kt2pp',
 'LCTSt3ipp',
 'LYSt2pp',
 'MACPD',
 'MALS',
 'MALt2_2pp',
 'MALt2_3pp',
 'MINCYCtpp',
 'ME1',
 'ME2',
 'MLTG3',
 'MLTG4',
 'MLTG5',
 'MN2tpp',
 'MNt2pp',
 'MG2tpp',
 'NADTRHD',
 'NAt3_1p5pp',
 'NAt3_2pp',
 'NOVBCNtpp',
 'NI2t3pp',
 'NI2tpp',
 'NMNAT',
 'OAADC',
 'PFL',
 'PTRCt2pp',
 'RFAMPtpp',
 'PUNP1',
 'PUNP5',
 'PYK',
 'PYNP2r',
 'SSALx',
 'SSALy',
 'SUCASPtpp',
 'SUCCt2_2pp',
 'SUCCt2_3pp',
 'SUCMALtpp',
 'TALA',
 'THD2pp',
 'THRA2',
 'THRt4pp',
 'TRE6PH',
 'TTRCYCtpp',
 'XANt2pp',
 'XYLI2',
 'ZN2t3pp',
 'ZN2tpp',
 'PPCK',

In [27]:
i=0
one_to_three_data_point_below_zero_and_one_to_three_data_point_above_zero= list()

if len(under_below_zero_and_under_above_zero)> 0:
    
    for r_id in under_below_zero_and_under_above_zero:    #separates under_below_zero_and_under_above_zero reactions in more specific subcategories
        l= list()
        n=0
        n2=0
        count=0
        count2=0
        r_id = under_below_zero_and_under_above_zero[i]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))

        while n < int(((len(csvFile)-1)/2)+1):
            inf= csvFile.iloc[n,  2]
            flt= float(csvFile.iloc[n,  2])
    
            if csvFile.iloc[n,  2] == flt:
                l.append(csvFile.iloc[n,  2])
            n= n+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
    
    
        n= int((len(csvFile)-1)/2)
        l4= list()
    
        while n < len(csvFile):
            inf= csvFile.iloc[n,  2]
            flt= float(csvFile.iloc[n,  2])
    
            if csvFile.iloc[n,  2] == flt:
                l4.append(csvFile.iloc[n,  2])
            n= n+1

        csvFile3= pd.DataFrame (l4, columns = ['Biomass'])
        csvFile3.Biomass = csvFile3.Biomass.round(2)
    
    
        my_series = csvFile2['Biomass'].squeeze()
        my_series2 = csvFile3['Biomass'].squeeze()


        while n2 < len(csvFile2):

            if csvFile2.iloc[n2,  0] > max(csvFile2["Biomass"])-0.01 and csvFile2.iloc[n2,  0] < max(csvFile2["Biomass"])+0.01:
                count= count+1
    
            n2= n2+1
        
        n2=0
        while n2 < len(csvFile3):

            if csvFile3.iloc[n2,  0] > max(csvFile3["Biomass"])-0.01 and csvFile3.iloc[n2,  0] < max(csvFile3["Biomass"])+0.01:
                count2= count2+1
    
            n2= n2+1
    
        if len(csvFile2)<4 and len(csvFile3)<4:
            one_to_three_data_point_below_zero_and_one_to_three_data_point_above_zero.append(r_id)

        elif len(csvFile2)<4 and (count2> 2 and count2< len(csvFile3)-3):
            one_to_three_data_point_below_zero_and_indecr_and_stable_above_zero.append(r_id)
            
        elif (count> 2 and count< len(csvFile2)-3) and len(csvFile3)<4:       
            indecr_and_stable_below_zero_and_one_to_three_data_point_above_zero.append(r_id)
        
        elif (count> len(csvFile2)-3 and len(csvFile2)>5) and len(csvFile3)<4:       
            flat_below_zero_and_one_to_three_data_point_above_zero.append(r_id)
        
        elif (count> 2 and len(csvFile2) < 6) and len(csvFile3)<4:       
            flat_below_zero_and_one_to_three_data_point_above_zero.append(r_id)
        
        elif (count> 1 and len(csvFile2) < 3) and len(csvFile3)<4:       
            flat_below_zero_and_one_to_three_data_point_above_zero.append(r_id)
        
            
    
        elif (count> 2 and count< len(csvFile2)-3) and (count2> 2 and count2< len(csvFile3)-3):        
            indecr_and_stable_below_zero_and_indecr_and_stable_above_zero.append(r_id)
        
        elif (count> len(csvFile2)-3 and len(csvFile2)>5) and (count2> 2 and count2< len(csvFile3)-3):
            flat_below_zero_and_indecr_and_stable_above_zero.append(r_id)
    
        elif (count> 2 and len(csvFile2) < 6) and (count2> 2 and count2< len(csvFile3)-3):
            flat_below_zero_and_indecr_and_stable_above_zero.append(r_id)
    
        elif (count> 1 and len(csvFile2) < 3) and (count2> 2 and count2< len(csvFile3)-3):
            flat_below_zero_and_indecr_and_stable_above_zero.append(r_id)
    
        elif count< 3 and (count2> 2 and count2< len(csvFile3)-3):
            diff_indecr_below_zero_and_indecr_and_stable_above_zero.append(r_id)
    
    
    
        elif len(csvFile2)<4 and (count2> len(csvFile3)-3 and len(csvFile3)>5):
            one_to_three_data_point_below_zero_and_flat_above_zero.append(r_id)
    
        elif len(csvFile2)<4 and (count2> 2 and len(csvFile3) < 6):
            one_to_three_data_point_below_zero_and_flat_above_zero.append(r_id)
    
        elif len(csvFile2)<4 and (count2> 1 and len(csvFile3) < 3):
            one_to_three_data_point_below_zero_and_flat_above_zero.append(r_id)
    
        elif (count> 2 and count< len(csvFile2)-3) and (count2> len(csvFile3)-3 and len(csvFile3)>5):
            indecr_and_stable_below_zero_and_flat_above_zero.append(r_id)
    
        elif (count> 2 and count< len(csvFile2)-3) and (count2> 2 and len(csvFile3) < 6):
            indecr_and_stable_below_zero_and_flat_above_zero.append(r_id)
    
        elif (count> 2 and count< len(csvFile2)-3) and (count2> 1 and len(csvFile3) < 3):
            indecr_and_stable_below_zero_and_flat_above_zero.append(r_id)
    
        elif count< 3 and (count2> len(csvFile3)-3 and len(csvFile3)>5):
            diff_indecr_below_zero_and_flat_above_zero.append(r_id)
    
        elif count< 3 and (count2> 2 and len(csvFile3) < 6):
            diff_indecr_below_zero_and_flat_above_zero.append(r_id)
    
        elif count< 3 and (count2> 1 and len(csvFile3) < 3):
            diff_indecr_below_zero_and_flat_above_zero.append(r_id)

        elif (count> len(csvFile2)-3 and len(csvFile2)>5) and (count2> len(csvFile3)-3 and len(csvFile3)>5):
            flat_below_zero_and_flat_above_zero.append(r_id)
    
        elif (count> len(csvFile2)-3 and len(csvFile2)>5) and (count2> 2 and len(csvFile3) < 6):
            flat_below_zero_and_flat_above_zero.append(r_id)
        
        elif (count> len(csvFile2)-3 and len(csvFile2)>5) and (count2> 1 and len(csvFile3) < 3):
            flat_below_zero_and_flat_above_zero.append(r_id)
    
        elif (count> 2 and len(csvFile2) < 6) and (count2> len(csvFile3)-3 and len(csvFile3)>5):
            flat_below_zero_and_flat_above_zero.append(r_id)
    
        elif (count> 2 and len(csvFile2) < 6) and (count2> 2 and len(csvFile3) < 6):
            flat_below_zero_and_flat_above_zero.append(r_id)
    
        elif (count> 2 and len(csvFile2) < 6) and (count2> 1 and len(csvFile3) < 3):
            flat_below_zero_and_flat_above_zero.append(r_id)
    
        elif (count> 1 and len(csvFile2) < 3) and (count2> len(csvFile3)-3 and len(csvFile3)>5):
            flat_below_zero_and_flat_above_zero.append(r_id)

        elif (count> 1 and len(csvFile2) < 3) and (count2> 2 and len(csvFile3) < 6):
            flat_below_zero_and_flat_above_zero.append(r_id)
    
        elif (count> 1 and len(csvFile2) < 3) and (count2> 1 and len(csvFile3) < 3):
            flat_below_zero_and_flat_above_zero.append(r_id)
    
    
    
        elif len(csvFile2)<4 and count2< 3:
            one_to_three_data_point_below_zero_and_diff_indecr_above_zero.append(r_id)
        
        elif count< 3 and len(csvFile3)<4:
            diff_indecr_below_zero_and_one_to_three_data_point_above_zero.append(r_id)
    
        elif (count> 2 and count< len(csvFile2)-3) and count2< 3:
            indecr_and_stable_below_zero_and_diff_indecr_above_zero.append(r_id)
        
        elif count< 3 and count2< 3:
            diff_indecr_below_zero_and_diff_indecr_above_zero.append(r_id)

        elif (count> len(csvFile2)-3 and len(csvFile2)>5) and count2< 3:
            flat_below_zero_and_diff_indecr_above_zero.append(r_id)
    
        elif (count> 2 and len(csvFile2) < 6) and count2< 3:
            flat_below_zero_and_diff_indecr_above_zero.append(r_id)
    
        elif (count> 1 and len(csvFile2) < 3) and count2< 3:
            flat_below_zero_and_diff_indecr_above_zero.append(r_id)
    
    
        i= i+1

indecr_and_stable_below_zero_and_indecr_and_stable_above_zero


['ACALD',
 'ALDD2y',
 'ASPt2_2pp',
 'ASPt2_3pp',
 'CITL',
 'CMtpp',
 'CD2t3pp',
 'COBALT2t3pp',
 'CYStpp',
 'CYTBD2pp',
 'CYTBDpp',
 'CYTBO3_4pp',
 'ARBt2rpp',
 'DAAD',
 'ARBt3ipp',
 'DOXRBCNtpp',
 'F6PP',
 'FADRx',
 'FADRx2',
 'FBA',
 'FBP',
 'FE2t2pp',
 'ASNS2',
 'FLDR2',
 'FE2tpp',
 'FORt2pp',
 'FUSAtpp',
 'G6PP',
 'GLCptspp',
 'GLCt2pp',
 'ASPT',
 'GDPDPK',
 'GDPMNH',
 'GDPTPDP',
 'GUAt2pp',
 'GLYCLTDx',
 'GLYCLTDy',
 'IDOND2',
 'GLYK',
 'HPYRRy',
 'Kt2pp',
 'LCTSt3ipp',
 'LYSt2pp',
 'MACPD',
 'MALS',
 'MALt2_2pp',
 'MALt2_3pp',
 'MINCYCtpp',
 'ME1',
 'ME2',
 'MLTG3',
 'MLTG4',
 'MLTG5',
 'MN2tpp',
 'MNt2pp',
 'MG2tpp',
 'NADTRHD',
 'NAt3_1p5pp',
 'NAt3_2pp',
 'NOVBCNtpp',
 'NI2t3pp',
 'NI2tpp',
 'NMNAT',
 'OAADC',
 'PFL',
 'PTRCt2pp',
 'RFAMPtpp',
 'PUNP1',
 'PUNP5',
 'PYK',
 'PYNP2r',
 'SSALx',
 'SSALy',
 'SUCASPtpp',
 'SUCCt2_2pp',
 'SUCCt2_3pp',
 'SUCMALtpp',
 'TALA',
 'THD2pp',
 'THRA2',
 'THRt4pp',
 'TRE6PH',
 'TTRCYCtpp',
 'XANt2pp',
 'XYLI2',
 'ZN2t3pp',
 'ZN2tpp',
 'PPCK',

In [28]:
flat_below_zero_and_flat_above_zero
indecr_and_stable_below_zero_and_one_to_three_data_point_above_zero
indecr_and_stable_below_zero_and_diff_indecr_above_zero
one_to_three_data_point_below_zero_and_indecr_and_stable_above_zero
one_to_three_data_point_below_zero_and_flat_above_zero
one_to_three_data_point_below_zero_and_diff_indecr_above_zero
one_to_three_data_point_below_zero_and_one_to_three_data_point_above_zero
diff_indecr_below_zero_and_diff_indecr_above_zero
diff_indecr_below_zero_and_indecr_and_stable_above_zero
diff_indecr_below_zero_and_one_to_three_data_point_above_zero
flat_below_zero_and_diff_indecr_above_zero
flat_below_zero_and_indecr_and_stable_above_zero
flat_below_zero_and_one_to_three_data_point_above_zero
diff_indecr_below_zero_and_flat_above_zero


indecr_and_stable_below_zero_and_indecr_and_stable_above_zero
indecr_and_stable_below_zero_and_flat_above_zero
one_to_three_data_point_above_zero
one_to_three_data_point_below_zero


['EX_k_e',
 'PAPA120pp',
 'PAPA140pp',
 'PAPA141pp',
 'PAPA160pp',
 'PAPA180pp',
 'UDCPDPpp']

In [29]:
len(indecr_and_stable_below_zero_and_indecr_and_stable_above_zero)+len(indecr_and_stable_below_zero_and_flat_above_zero)+len(one)+len(zero)+len(flat_below_zero_and_flat_above_zero)+len(indecr_and_stable_below_zero_and_one_to_three_data_point_above_zero)+len(indecr_and_stable_below_zero_and_diff_indecr_above_zero)+len(one_to_three_data_point_below_zero_and_indecr_and_stable_above_zero)+len(one_to_three_data_point_below_zero_and_flat_above_zero)+len(one_to_three_data_point_below_zero_and_diff_indecr_above_zero)+len(one_to_three_data_point_below_zero_and_one_to_three_data_point_above_zero)+len(diff_indecr_below_zero_and_diff_indecr_above_zero)+len(diff_indecr_below_zero_and_indecr_and_stable_above_zero)+len(diff_indecr_below_zero_and_one_to_three_data_point_above_zero)+len(flat_below_zero_and_diff_indecr_above_zero)+len(flat_below_zero_and_indecr_and_stable_above_zero)+len(flat_below_zero_and_one_to_three_data_point_above_zero)+len(diff_indecr_below_zero_and_flat_above_zero)+len(one_at_zero)+len(mon_decr_and_stable_above_zero)+len(mon_incr_and_stable_above_zero)+len(inANDdecr_and_stable_above_zero)+len(mon_decr_and_stable_below_zero)+len(mon_incr_and_stable_below_zero)+len(inANDdecr_and_stable_below_zero)+len(flat_above_zero)+len(flat_below_zero)+len(diff_indecr_above_zero)+len(diff_indecr_below_zero)+len(one_to_three_data_point_above_zero)+len(one_to_three_data_point_below_zero)


2726

In [30]:
lt= list(indecr_and_stable_below_zero_and_indecr_and_stable_above_zero)+list(indecr_and_stable_below_zero_and_flat_above_zero)+list(one)+list(zero)+list(flat_below_zero_and_flat_above_zero)+list(indecr_and_stable_below_zero_and_one_to_three_data_point_above_zero)+list(indecr_and_stable_below_zero_and_diff_indecr_above_zero)+list(one_to_three_data_point_below_zero_and_indecr_and_stable_above_zero)+list(one_to_three_data_point_below_zero_and_flat_above_zero)+list(one_to_three_data_point_below_zero_and_diff_indecr_above_zero)+list(one_to_three_data_point_below_zero_and_one_to_three_data_point_above_zero)+list(diff_indecr_below_zero_and_diff_indecr_above_zero)+list(diff_indecr_below_zero_and_indecr_and_stable_above_zero)+list(diff_indecr_below_zero_and_one_to_three_data_point_above_zero)+list(flat_below_zero_and_diff_indecr_above_zero)+list(flat_below_zero_and_indecr_and_stable_above_zero)+list(flat_below_zero_and_one_to_three_data_point_above_zero)+list(diff_indecr_below_zero_and_flat_above_zero)+list(one_at_zero)+list(mon_decr_and_stable_above_zero)+list(mon_incr_and_stable_above_zero)+list(inANDdecr_and_stable_above_zero)+list(mon_decr_and_stable_below_zero)+list(mon_incr_and_stable_below_zero)+list(inANDdecr_and_stable_below_zero)+list(flat_above_zero)+list(flat_below_zero)+list(diff_indecr_above_zero)+list(diff_indecr_below_zero)+list(one_to_three_data_point_above_zero)+list(one_to_three_data_point_below_zero)


In [31]:
l1=[]
for i in lt:
    if i not in l1:
        l1.append(i)
    else:
        print(i,end=' ')
        

In [32]:
csvFile = pd.read_csv('all reactions.csv')
csvFile["Reaction names"]

rl= list(csvFile["Reaction names"])

set1= set(rl)
set2= set(lt)
set3= set1-set2
set3


set()

In [33]:
i2= 0
incr_and_stable_below_zero_and_one_to_three_data_point_above_zero= list()
decr_and_stable_below_zero_and_one_to_three_data_point_above_zero= list()

if len(indecr_and_stable_below_zero_and_one_to_three_data_point_above_zero)>0:
    
    for r_id in indecr_and_stable_below_zero_and_one_to_three_data_point_above_zero:                    #this code finds which of the indecr_and_stable_below_zero_and_one_to_three_data_point_above_zero reactions have an increasing and stable trend and which a decreasing and stable trend below zero
    
        i=0
        r_id = indecr_and_stable_below_zero_and_one_to_three_data_point_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        l= list()

        while i < ((len(csvFile)-1)/2)+1:
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)

        my_series = csvFile2['Biomass'].squeeze()
        type(my_series)

        if my_series.is_monotonic_decreasing ==True:
            decr_and_stable_below_zero_and_one_to_three_data_point_above_zero.append(r_id)
        elif my_series.is_monotonic ==True:
            incr_and_stable_below_zero_and_one_to_three_data_point_above_zero.append(r_id)

    
        i2= i2 + 1

decr_and_stable_below_zero_and_one_to_three_data_point_above_zero

set1= set(indecr_and_stable_below_zero_and_one_to_three_data_point_above_zero)
set2= set(decr_and_stable_below_zero_and_one_to_three_data_point_above_zero)
set3= set(incr_and_stable_below_zero_and_one_to_three_data_point_above_zero)
inANDdecr_and_stable_below_zero_and_one_to_three_data_point_above_zero= list(set1-set2-set3)
inANDdecr_and_stable_below_zero_and_one_to_three_data_point_above_zero


[]

In [34]:
w3= list()
w4= list()
i2=0

if len(inANDdecr_and_stable_below_zero_and_one_to_three_data_point_above_zero)>0:
    
    while i2 < len(inANDdecr_and_stable_below_zero_and_one_to_three_data_point_above_zero):       #this code finds inANDdecr_and_stable_below_zero_and_one_to_three_data_point_above_zero reactions that are not truly both increasing and decreasing and stable below zero
    
        i=0
        r_id = inANDdecr_and_stable_below_zero_and_one_to_three_data_point_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        l= list()

        while i < int(((len(csvFile)-1)/2)+1):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and (l[n2+3] < max(l) or l[n2+3] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

    
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and (l[n2-3] < max(l) or l[n2-3] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
    
        i2= i2+1

w3


[]

In [35]:
union = pd.Series(np.union1d(w3, w4))
unclassified_below_zero_and_one_to_three_data_point_above_zero = pd.Series(np.intersect1d(w3, w4))

set1= set(w3)
set2= set(unclassified_below_zero_and_one_to_three_data_point_above_zero)
set3= set1-set2
w3= list(set3)

set1= set(w4)
set2= set(unclassified_below_zero_and_one_to_three_data_point_above_zero)
set3= set1-set2
w4= list(set3)

unclassified_below_zero_and_one_to_three_data_point_above_zero= list(unclassified_below_zero_and_one_to_three_data_point_above_zero)
unclassified_below_zero_and_one_to_three_data_point_above_zero


[]

In [36]:
set1= set(inANDdecr_and_stable_below_zero_and_one_to_three_data_point_above_zero)
set2= set(w3)
set3= set(w4)
set4= set(unclassified_below_zero_and_one_to_three_data_point_above_zero)
set5= set1-set2-set3-set4
inANDdecr_and_stable_below_zero_and_one_to_three_data_point_above_zero= list(set5)

decr_and_stable_below_zero_and_one_to_three_data_point_above_zero= list(decr_and_stable_below_zero_and_one_to_three_data_point_above_zero) + list(w3)
incr_and_stable_below_zero_and_one_to_three_data_point_above_zero= list(incr_and_stable_below_zero_and_one_to_three_data_point_above_zero) + list(w4)


In [37]:
i2= 0
indecr_and_stable_below_zero_and_mon_incr_above_zero= list()
indecr_and_stable_below_zero_and_mon_decr_above_zero= list()

if len(indecr_and_stable_below_zero_and_diff_indecr_above_zero)>0:
    
    for r_id in indecr_and_stable_below_zero_and_diff_indecr_above_zero:                  #this code finds which of the indecr_and_stable_below_zero_and_diff_indecr_above_zero reactions have an increasing trend and which a decreasing trend above zero
    
        r_id = indecr_and_stable_below_zero_and_diff_indecr_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        n2= int((len(csvFile)-1)/2)
        l= list()
    
        while n2 < len(csvFile):
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                indecr_and_stable_below_zero_and_mon_decr_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                indecr_and_stable_below_zero_and_mon_incr_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(indecr_and_stable_below_zero_and_diff_indecr_above_zero)
set2= set(indecr_and_stable_below_zero_and_mon_decr_above_zero)
set3= set(indecr_and_stable_below_zero_and_mon_incr_above_zero)
indecr_and_stable_below_zero_and_inANDdecr_above_zero= list(set1-set2-set3)
indecr_and_stable_below_zero_and_inANDdecr_above_zero


[]

In [38]:
w3= list()
w4= list()
i2=0

if len(indecr_and_stable_below_zero_and_inANDdecr_above_zero)>0:
    
    while i2 < len(indecr_and_stable_below_zero_and_inANDdecr_above_zero):       #this code finds indecr_and_stable_below_zero_and_inANDdecr_above_zero reactions that are not truly both increasing and decreasing and stable above zero
    
        i= int((len(csvFile)-1)/2)
        r_id = indecr_and_stable_below_zero_and_inANDdecr_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        l= list()

        while i < len(csvFile):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        
        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and (l[n2+3] < max(l) or l[n2+3] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

    
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and (l[n2-3] < max(l) or l[n2-3] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)

    
        i2= i2+1

w3


[]

In [39]:
union = pd.Series(np.union1d(w3, w4))
indecr_and_stable_below_zero_and_unclassified_above_zero = pd.Series(np.intersect1d(w3, w4))

set1= set(w3)
set2= set(indecr_and_stable_below_zero_and_unclassified_above_zero)
set3= set1-set2
w3= list(set3)

set1= set(w4)
set2= set(indecr_and_stable_below_zero_and_unclassified_above_zero)
set3= set1-set2
w4= list(set3)

indecr_and_stable_below_zero_and_unclassified_above_zero= list(indecr_and_stable_below_zero_and_unclassified_above_zero)
indecr_and_stable_below_zero_and_unclassified_above_zero


[]

In [40]:
set1= set(indecr_and_stable_below_zero_and_inANDdecr_above_zero)
set2= set(w3)
set3= set(w4)
set4= set(indecr_and_stable_below_zero_and_unclassified_above_zero)
set5= set1-set2-set3-set4
indecr_and_stable_below_zero_and_inANDdecr_above_zero= list(set5)

indecr_and_stable_below_zero_and_mon_decr_above_zero= list(indecr_and_stable_below_zero_and_mon_decr_above_zero) + list(w3)
indecr_and_stable_below_zero_and_mon_incr_above_zero= list(indecr_and_stable_below_zero_and_mon_incr_above_zero) + list(w4)


In [41]:
i2= 0
decr_and_stable_below_zero_and_mon_decr_above_zero= list()
incr_and_stable_below_zero_and_mon_decr_above_zero= list()

if len(indecr_and_stable_below_zero_and_mon_decr_above_zero)>0:
    
    for r_id in indecr_and_stable_below_zero_and_mon_decr_above_zero:                #this code finds which of the indecr_and_stable_below_zero_and_mon_decr_above_zero reactions have an increasing and stable trend and which a decreasing and stable trend below zero
    
        r_id = indecr_and_stable_below_zero_and_mon_decr_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        n2= 0
        l= list()
    
        while n2 < ((len(csvFile)-1)/2)+1:
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                decr_and_stable_below_zero_and_mon_decr_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                incr_and_stable_below_zero_and_mon_decr_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(indecr_and_stable_below_zero_and_mon_decr_above_zero)
set2= set(decr_and_stable_below_zero_and_mon_decr_above_zero)
set3= set(incr_and_stable_below_zero_and_mon_decr_above_zero)
inANDdecr_and_stable_below_zero_and_mon_decr_above_zero= list(set1-set2-set3)
inANDdecr_and_stable_below_zero_and_mon_decr_above_zero


['CO2tpp',
 'ACOAD1f',
 'PGM',
 'ACOAD2f',
 'ACOAD3f',
 'RPI',
 'PGK',
 'GLYCDx',
 'GLUDy']

In [42]:
w3= list()
w4= list()
i2=0

if len(inANDdecr_and_stable_below_zero_and_mon_decr_above_zero)>0:
    
    while i2 < len(inANDdecr_and_stable_below_zero_and_mon_decr_above_zero):       #this code finds inANDdecr_and_stable_below_zero_and_mon_decr_above_zero reactions that are not truly both increasing and decreasing and stable below zero
    
        i=0
        r_id = inANDdecr_and_stable_below_zero_and_mon_decr_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        l= list()

        while i < ((len(csvFile)-1)/2)+1:
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and (l[n2+3] < max(l) or l[n2+3] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

    
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and (l[n2-3] < max(l) or l[n2-3] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
            
        i2= i2+1

w3


[]

In [43]:
union = pd.Series(np.union1d(w3, w4))
unclassified_below_zero_and_mon_decr_above_zero = pd.Series(np.intersect1d(w3, w4))

set1= set(w3)
set2= set(unclassified_below_zero_and_mon_decr_above_zero)
set3= set1-set2
w3= list(set3)

set1= set(w4)
set2= set(unclassified_below_zero_and_mon_decr_above_zero)
set3= set1-set2
w4= list(set3)

unclassified_below_zero_and_mon_decr_above_zero= list(unclassified_below_zero_and_mon_decr_above_zero)
unclassified_below_zero_and_mon_decr_above_zero


[]

In [44]:
w4

set1= set(inANDdecr_and_stable_below_zero_and_mon_decr_above_zero)
set2= set(w3)
set3= set(w4)
set4= set(unclassified_below_zero_and_mon_decr_above_zero)
set5= set1-set2-set3-set4
inANDdecr_and_stable_below_zero_and_mon_decr_above_zero= list(set5)

decr_and_stable_below_zero_and_mon_decr_above_zero= list(decr_and_stable_below_zero_and_mon_decr_above_zero) + list(w3)
incr_and_stable_below_zero_and_mon_decr_above_zero= list(incr_and_stable_below_zero_and_mon_decr_above_zero) + list(w4)


In [45]:
i2= 0
decr_and_stable_below_zero_and_mon_incr_above_zero= list()
incr_and_stable_below_zero_and_mon_incr_above_zero= list()

if len(indecr_and_stable_below_zero_and_mon_incr_above_zero)> 0:

    for r_id in indecr_and_stable_below_zero_and_mon_incr_above_zero:              #this code finds which of the indecr_and_stable_below_zero_and_mon_incr_above_zero reactions have an increasing and stable trend and which a decreasing and stable trend below zero
    
        r_id = indecr_and_stable_below_zero_and_mon_incr_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        n2= 0
        l= list()
    
        while n2 < ((len(csvFile)-1)/2)+1:
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                decr_and_stable_below_zero_and_mon_incr_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                incr_and_stable_below_zero_and_mon_incr_above_zero.append(r_id)

    
        i2= i2 + 1


set1= set(indecr_and_stable_below_zero_and_mon_incr_above_zero)
set2= set(decr_and_stable_below_zero_and_mon_incr_above_zero)
set3= set(incr_and_stable_below_zero_and_mon_incr_above_zero)
inANDdecr_and_stable_below_zero_and_mon_incr_above_zero= list(set1-set2-set3)
inANDdecr_and_stable_below_zero_and_mon_incr_above_zero


[]

In [46]:
w3= list()
w4= list()
i2=0

if len(inANDdecr_and_stable_below_zero_and_mon_incr_above_zero)>0:
    
    while i2 < len(inANDdecr_and_stable_below_zero_and_mon_incr_above_zero):       #this code finds inANDdecr_and_stable_below_zero_and_mon_incr_above_zero reactions that are not truly both increasing and decreasing and stable below zero
    
        i=0
        r_id = inANDdecr_and_stable_below_zero_and_mon_incr_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        l= list()

        while i < ((len(csvFile)-1)/2)+1:
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and (l[n2+3] < max(l) or l[n2+3] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

    
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and (l[n2-3] < max(l) or l[n2-3] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
            
        i2= i2+1

w3


[]

In [47]:
union = pd.Series(np.union1d(w3, w4))
unclassified_below_zero_and_mon_incr_above_zero = pd.Series(np.intersect1d(w3, w4))

set1= set(w3)
set2= set(unclassified_below_zero_and_mon_incr_above_zero)
set3= set1-set2
w3= list(set3)

set1= set(w4)
set2= set(unclassified_below_zero_and_mon_incr_above_zero)
set3= set1-set2
w4= list(set3)

unclassified_below_zero_and_mon_incr_above_zero= list(unclassified_below_zero_and_mon_incr_above_zero)
unclassified_below_zero_and_mon_incr_above_zero


[]

In [48]:
w4

set1= set(inANDdecr_and_stable_below_zero_and_mon_incr_above_zero)
set2= set(w3)
set3= set(w4)
set4= set(unclassified_below_zero_and_mon_incr_above_zero)
set5= set1-set2-set3-set4
inANDdecr_and_stable_below_zero_and_mon_incr_above_zero= list(set5)

decr_and_stable_below_zero_and_mon_incr_above_zero= list(decr_and_stable_below_zero_and_mon_incr_above_zero) + list(w3)
incr_and_stable_below_zero_and_mon_incr_above_zero= list(incr_and_stable_below_zero_and_mon_incr_above_zero) + list(w4)


In [49]:
i2= 0
decr_and_stable_below_zero_and_inANDdecr_above_zero= list()
incr_and_stable_below_zero_and_inANDdecr_above_zero= list()

if len(indecr_and_stable_below_zero_and_inANDdecr_above_zero)> 0:

    for r_id in indecr_and_stable_below_zero_and_inANDdecr_above_zero:           #this code finds which of the indecr_and_stable_below_zero_and_inANDdecr_above_zero reactions have an increasing and stable trend and which a decreasing and stable trend below zero
    
        r_id = indecr_and_stable_below_zero_and_inANDdecr_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        n2= 0
        l= list()
    
        while n2 < ((len(csvFile)-1)/2)+1:
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)


        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                decr_and_stable_below_zero_and_inANDdecr_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                incr_and_stable_below_zero_and_inANDdecr_above_zero.append(r_id)
    
        i2= i2 + 1


set1= set(indecr_and_stable_below_zero_and_inANDdecr_above_zero)
set2= set(decr_and_stable_below_zero_and_inANDdecr_above_zero)
set3= set(incr_and_stable_below_zero_and_inANDdecr_above_zero)
inANDdecr_and_stable_below_zero_and_inANDdecr_above_zero= list(set1-set2-set3)
inANDdecr_and_stable_below_zero_and_inANDdecr_above_zero


[]

In [50]:
w3= list()
w4= list()
i2=0

if len(inANDdecr_and_stable_below_zero_and_inANDdecr_above_zero)>0:
    
    while i2 < len(inANDdecr_and_stable_below_zero_and_inANDdecr_above_zero):       #this code finds inANDdecr_and_stable_below_zero_and_inANDdecr_above_zero reactions that are not truly both increasing and decreasing and stable below zero
    
        i=0
        r_id = inANDdecr_and_stable_below_zero_and_inANDdecr_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        l= list()

        while i < ((len(csvFile)-1)/2)+1:
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and (l[n2+3] < max(l) or l[n2+3] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

    
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and (l[n2-3] < max(l) or l[n2-3] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
            
        i2= i2+1

w3


[]

In [51]:
union = pd.Series(np.union1d(w3, w4))
unclassified_below_zero_and_inANDdecr_above_zero = pd.Series(np.intersect1d(w3, w4))

set1= set(w3)
set2= set(unclassified_below_zero_and_inANDdecr_above_zero)
set3= set1-set2
w3= list(set3)

set1= set(w4)
set2= set(unclassified_below_zero_and_inANDdecr_above_zero)
set3= set1-set2
w4= list(set3)

unclassified_below_zero_and_inANDdecr_above_zero= list(unclassified_below_zero_and_inANDdecr_above_zero)
unclassified_below_zero_and_inANDdecr_above_zero


[]

In [52]:
w4

set1= set(inANDdecr_and_stable_below_zero_and_inANDdecr_above_zero)
set2= set(w3)
set3= set(w4)
set4= set(unclassified_below_zero_and_inANDdecr_above_zero)
set5= set1-set2-set3-set4
inANDdecr_and_stable_below_zero_and_inANDdecr_above_zero= list(set5)

decr_and_stable_below_zero_and_mon_incr_above_zero= list(decr_and_stable_below_zero_and_mon_incr_above_zero) + list(w3)
incr_and_stable_below_zero_and_mon_incr_above_zero= list(incr_and_stable_below_zero_and_mon_incr_above_zero) + list(w4)


In [53]:
i2= 0
one_to_three_data_point_below_zero_and_incr_and_stable_above_zero= list()
one_to_three_data_point_below_zero_and_decr_and_stable_above_zero= list()

if len(one_to_three_data_point_below_zero_and_indecr_and_stable_above_zero)>0:
    
    for r_id in one_to_three_data_point_below_zero_and_indecr_and_stable_above_zero:
    
        r_id = one_to_three_data_point_below_zero_and_indecr_and_stable_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        n2= int((len(csvFile)-1)/2)
        l= list()
    
        while n2 < len(csvFile):
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                one_to_three_data_point_below_zero_and_decr_and_stable_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                one_to_three_data_point_below_zero_and_incr_and_stable_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(one_to_three_data_point_below_zero_and_indecr_and_stable_above_zero)
set2= set(one_to_three_data_point_below_zero_and_decr_and_stable_above_zero)
set3= set(one_to_three_data_point_below_zero_and_incr_and_stable_above_zero)
one_to_three_data_point_below_zero_and_inANDdecr_and_stable_above_zero= list(set1-set2-set3)
one_to_three_data_point_below_zero_and_inANDdecr_and_stable_above_zero


[]

In [54]:
w3= list()
w4= list()
i2=0

if len(one_to_three_data_point_below_zero_and_inANDdecr_and_stable_above_zero)>0:
    
    while i2 < len(one_to_three_data_point_below_zero_and_inANDdecr_and_stable_above_zero):
    
        i= int((len(csvFile)-1)/2)
        r_id = one_to_three_data_point_below_zero_and_inANDdecr_and_stable_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        l= list()

        while i < len(csvFile):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and (l[n2+3] < max(l) or l[n2+3] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

    
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and (l[n2-3] < max(l) or l[n2-3] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
            
        i2= i2+1

w3


[]

In [55]:
union = pd.Series(np.union1d(w3, w4))
one_to_three_data_point_below_zero_and_unclassified_above_zero = pd.Series(np.intersect1d(w3, w4))

set1= set(w3)
set2= set(one_to_three_data_point_below_zero_and_unclassified_above_zero)
set3= set1-set2
w3= list(set3)

set1= set(w4)
set2= set(one_to_three_data_point_below_zero_and_unclassified_above_zero)
set3= set1-set2
w4= list(set3)

one_to_three_data_point_below_zero_and_unclassified_above_zero= list(one_to_three_data_point_below_zero_and_unclassified_above_zero)
one_to_three_data_point_below_zero_and_unclassified_above_zero


[]

In [56]:
w4

set1= set(one_to_three_data_point_below_zero_and_inANDdecr_and_stable_above_zero)
set2= set(w3)
set3= set(w4)
set4= set(one_to_three_data_point_below_zero_and_unclassified_above_zero)
set5= set1-set2-set3-set4
one_to_three_data_point_below_zero_and_inANDdecr_and_stable_above_zero= list(set5)

one_to_three_data_point_below_zero_and_decr_and_stable_above_zero= list(one_to_three_data_point_below_zero_and_decr_and_stable_above_zero) + list(w3)
one_to_three_data_point_below_zero_and_incr_and_stable_above_zero= list(one_to_three_data_point_below_zero_and_incr_and_stable_above_zero) + list(w4)


In [57]:
i2= 0
one_to_three_data_point_below_zero_and_mon_incr_above_zero= list()
one_to_three_data_point_below_zero_and_mon_decr_above_zero= list()

if len(one_to_three_data_point_below_zero_and_diff_indecr_above_zero)>0:
    
    for r_id in one_to_three_data_point_below_zero_and_diff_indecr_above_zero:
    
        r_id = one_to_three_data_point_below_zero_and_diff_indecr_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        n2= int((len(csvFile)-1)/2)
        l= list()
    
        while n2 < len(csvFile):
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                one_to_three_data_point_below_zero_and_mon_decr_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                one_to_three_data_point_below_zero_and_mon_incr_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(one_to_three_data_point_below_zero_and_diff_indecr_above_zero)
set2= set(one_to_three_data_point_below_zero_and_mon_decr_above_zero)
set3= set(one_to_three_data_point_below_zero_and_mon_incr_above_zero)
one_to_three_data_point_below_zero_and_inANDdecr_above_zero= list(set1-set2-set3)
one_to_three_data_point_below_zero_and_inANDdecr_above_zero


[]

In [58]:
w3= list()
w4= list()
i2=0

if len(one_to_three_data_point_below_zero_and_inANDdecr_above_zero)>0:
    
    while i2 < len(one_to_three_data_point_below_zero_and_inANDdecr_above_zero):
    
        i= int((len(csvFile)-1)/2)
        r_id = one_to_three_data_point_below_zero_and_inANDdecr_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        l= list()

        while i < len(csvFile):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and (l[n2+3] < max(l) or l[n2+3] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

    
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and (l[n2-3] < max(l) or l[n2-3] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
            
        i2= i2+1

w3


[]

In [59]:
union = pd.Series(np.union1d(w3, w4))
one_to_three_data_point_below_zero_and_unclassified_above_zero2 = pd.Series(np.intersect1d(w3, w4))

set1= set(w3)
set2= set(one_to_three_data_point_below_zero_and_unclassified_above_zero2)
set3= set1-set2
w3= list(set3)

set1= set(w4)
set2= set(one_to_three_data_point_below_zero_and_unclassified_above_zero2)
set3= set1-set2
w4= list(set3)

one_to_three_data_point_below_zero_and_unclassified_above_zero= list(one_to_three_data_point_below_zero_and_unclassified_above_zero)+list(one_to_three_data_point_below_zero_and_unclassified_above_zero2)
one_to_three_data_point_below_zero_and_unclassified_above_zero


[]

In [60]:
w4

set1= set(one_to_three_data_point_below_zero_and_inANDdecr_above_zero)
set2= set(w3)
set3= set(w4)
set4= set(one_to_three_data_point_below_zero_and_unclassified_above_zero)
set5= set1-set2-set3-set4
one_to_three_data_point_below_zero_and_inANDdecr_above_zero= list(set5)

one_to_three_data_point_below_zero_and_mon_decr_above_zero= list(one_to_three_data_point_below_zero_and_mon_decr_above_zero) + list(w3)
one_to_three_data_point_below_zero_and_mon_incr_above_zero= list(one_to_three_data_point_below_zero_and_mon_incr_above_zero) + list(w4)


In [61]:
i2= 0
diff_indecr_below_zero_and_mon_incr_above_zero= list()
diff_indecr_below_zero_and_mon_decr_above_zero= list()

if len(diff_indecr_below_zero_and_diff_indecr_above_zero)>0:
    
    for r_id in diff_indecr_below_zero_and_diff_indecr_above_zero:
    
        r_id = diff_indecr_below_zero_and_diff_indecr_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        n2= int((len(csvFile)-1)/2)
        l= list()
    
        while n2 < len(csvFile):
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                diff_indecr_below_zero_and_mon_decr_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                diff_indecr_below_zero_and_mon_incr_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(diff_indecr_below_zero_and_diff_indecr_above_zero)
set2= set(diff_indecr_below_zero_and_mon_decr_above_zero)
set3= set(diff_indecr_below_zero_and_mon_incr_above_zero)
diff_indecr_below_zero_and_inANDdecr_above_zero= list(set1-set2-set3)
diff_indecr_below_zero_and_inANDdecr_above_zero


['EX_co2_e', 'GHMT2r']

In [62]:
w3= list()
w4= list()
i2=0

if len(diff_indecr_below_zero_and_inANDdecr_above_zero)>0:
    
    while i2 < len(diff_indecr_below_zero_and_inANDdecr_above_zero):
    
        i= int((len(csvFile)-1)/2)
        r_id = diff_indecr_below_zero_and_inANDdecr_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        l= list()

        while i < len(csvFile):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and (l[n2+3] < max(l) or l[n2+3] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

    
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and (l[n2-3] < max(l) or l[n2-3] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
            
        i2= i2+1

w3


[]

In [63]:
union = pd.Series(np.union1d(w3, w4))
one_to_three_data_point_below_zero_and_unclassified_above_zero2 = pd.Series(np.intersect1d(w3, w4))

set1= set(w3)
set2= set(one_to_three_data_point_below_zero_and_unclassified_above_zero2)
set3= set1-set2
w3= list(set3)

set1= set(w4)
set2= set(one_to_three_data_point_below_zero_and_unclassified_above_zero2)
set3= set1-set2
w4= list(set3)

one_to_three_data_point_below_zero_and_unclassified_above_zero= list(one_to_three_data_point_below_zero_and_unclassified_above_zero)+list(one_to_three_data_point_below_zero_and_unclassified_above_zero2)
one_to_three_data_point_below_zero_and_unclassified_above_zero


[]

In [64]:
w4

set1= set(diff_indecr_below_zero_and_inANDdecr_above_zero)
set2= set(w3)
set3= set(w4)
set4= set(one_to_three_data_point_below_zero_and_unclassified_above_zero)
set5= set1-set2-set3-set4
diff_indecr_below_zero_and_inANDdecr_above_zero= list(set5)

diff_indecr_below_zero_and_mon_decr_above_zero= list(diff_indecr_below_zero_and_mon_decr_above_zero) + list(w3)
diff_indecr_below_zero_and_mon_incr_above_zero= list(diff_indecr_below_zero_and_mon_incr_above_zero) + list(w4)


In [65]:
i2= 0
mon_decr_below_zero_and_mon_decr_above_zero= list()
mon_incr_below_zero_and_mon_decr_above_zero= list()

if len(diff_indecr_below_zero_and_mon_decr_above_zero)>0:
    
    for r_id in diff_indecr_below_zero_and_mon_decr_above_zero:
    
        r_id = diff_indecr_below_zero_and_mon_decr_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        n2= 0
        l= list()
    
        while n2 < int((len(csvFile)-1)/2+1):
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                mon_decr_below_zero_and_mon_decr_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                mon_incr_below_zero_and_mon_decr_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(diff_indecr_below_zero_and_mon_decr_above_zero)
set2= set(mon_decr_below_zero_and_mon_decr_above_zero)
set3= set(mon_incr_below_zero_and_mon_decr_above_zero)
inANDdecr_below_zero_and_mon_decr_above_zero= list(set1-set2-set3)
inANDdecr_below_zero_and_mon_decr_above_zero


['EX_val__L_e', 'EX_ura_e', 'EX_quin_e', 'CO2tex', 'EX_indole_e']

In [66]:
w3= list()
w4= list()
i2=0

if len(inANDdecr_below_zero_and_mon_decr_above_zero)>0:
    
    while i2 < len(inANDdecr_below_zero_and_mon_decr_above_zero):
    
        i= 0
        r_id = inANDdecr_below_zero_and_mon_decr_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        l= list()

        while i < int(((len(csvFile)-1)/2)+1):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and (l[n2+3] < max(l) or l[n2+3] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

    
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and (l[n2-3] < max(l) or l[n2-3] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
            
        i2= i2+1

        
        
w3


[]

In [67]:
w4

['EX_indole_e']

In [68]:
union = pd.Series(np.union1d(w3, w4))
unclassified_below_zero_and_mon_decr_above_zero2 = pd.Series(np.intersect1d(w3, w4))

set1= set(w3)
set2= set(unclassified_below_zero_and_mon_decr_above_zero2)
set3= set1-set2
w3= list(set3)

set1= set(w4)
set2= set(unclassified_below_zero_and_mon_decr_above_zero2)
set3= set1-set2
w4= list(set3)

unclassified_below_zero_and_mon_decr_above_zero= list(unclassified_below_zero_and_mon_decr_above_zero)+list(unclassified_below_zero_and_mon_decr_above_zero2)
unclassified_below_zero_and_mon_decr_above_zero


[]

In [69]:
w4

set1= set(inANDdecr_below_zero_and_mon_decr_above_zero)
set2= set(w3)
set3= set(w4)
set4= set(unclassified_below_zero_and_mon_decr_above_zero)
set5= set1-set2-set3-set4
inANDdecr_below_zero_and_mon_decr_above_zero= list(set5)

mon_decr_below_zero_and_mon_decr_above_zero= list(mon_decr_below_zero_and_mon_decr_above_zero) + list(w3)
mon_incr_below_zero_and_mon_decr_above_zero= list(mon_incr_below_zero_and_mon_decr_above_zero) + list(w4)


In [70]:
i2= 0
mon_decr_below_zero_and_mon_incr_above_zero= list()
mon_incr_below_zero_and_mon_incr_above_zero= list()

if len(diff_indecr_below_zero_and_mon_incr_above_zero)>0:
    
    for r_id in diff_indecr_below_zero_and_mon_incr_above_zero:
    
        r_id = diff_indecr_below_zero_and_mon_incr_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        n2= 0
        l= list()
    
        while n2 < int(((len(csvFile)-1)/2)+1):
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                mon_decr_below_zero_and_mon_incr_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                mon_incr_below_zero_and_mon_incr_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(diff_indecr_below_zero_and_mon_incr_above_zero)
set2= set(mon_decr_below_zero_and_mon_incr_above_zero)
set3= set(mon_incr_below_zero_and_mon_incr_above_zero)
inANDdecr_below_zero_and_mon_incr_above_zero= list(set1-set2-set3)
inANDdecr_below_zero_and_mon_incr_above_zero


[]

In [71]:
w3= list()
w4= list()
i2=0

if len(inANDdecr_below_zero_and_mon_incr_above_zero)>0:
    
    while i2 < len(inANDdecr_below_zero_and_mon_incr_above_zero):
    
        i= 0
        r_id = inANDdecr_below_zero_and_mon_incr_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        l= list()

        while i < int(((len(csvFile)-1)/2)+1):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and (l[n2+3] < max(l) or l[n2+3] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

    
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and (l[n2-3] < max(l) or l[n2-3] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
            
        i2= i2+1

w3


[]

In [72]:
union = pd.Series(np.union1d(w3, w4))
unclassified_below_zero_and_mon_incr_above_zero2 = pd.Series(np.intersect1d(w3, w4))

set1= set(w3)
set2= set(unclassified_below_zero_and_mon_incr_above_zero2)
set3= set1-set2
w3= list(set3)

set1= set(w4)
set2= set(unclassified_below_zero_and_mon_incr_above_zero2)
set3= set1-set2
w4= list(set3)

unclassified_below_zero_and_mon_incr_above_zero= list(unclassified_below_zero_and_mon_incr_above_zero)+list(unclassified_below_zero_and_mon_incr_above_zero2)
unclassified_below_zero_and_mon_incr_above_zero


[]

In [73]:
w4

set1= set(inANDdecr_below_zero_and_mon_incr_above_zero)
set2= set(w3)
set3= set(w4)
set4= set(unclassified_below_zero_and_mon_incr_above_zero)
set5= set1-set2-set3-set4
inANDdecr_below_zero_and_mon_incr_above_zero= list(set5)

mon_decr_below_zero_and_mon_incr_above_zero= list(mon_decr_below_zero_and_mon_incr_above_zero) + list(w3)
mon_incr_below_zero_and_mon_incr_above_zero= list(mon_incr_below_zero_and_mon_incr_above_zero) + list(w4)


In [74]:
i2= 0
mon_decr_below_zero_and_inANDdecr_above_zero= list()
mon_incr_below_zero_and_inANDdecr_above_zero= list()

if len(diff_indecr_below_zero_and_inANDdecr_above_zero)>0:
    
    for r_id in diff_indecr_below_zero_and_inANDdecr_above_zero:                  
        
        r_id = diff_indecr_below_zero_and_inANDdecr_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        n2= 0
        l= list()
    
        while n2 < int((len(csvFile)-1)/2+1):
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                mon_decr_below_zero_and_inANDdecr_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                mon_incr_below_zero_and_inANDdecr_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(diff_indecr_below_zero_and_inANDdecr_above_zero)
set2= set(mon_decr_below_zero_and_inANDdecr_above_zero)
set3= set(mon_incr_below_zero_and_inANDdecr_above_zero)
inANDdecr_below_zero_and_inANDdecr_above_zero= list(set1-set2-set3)
inANDdecr_below_zero_and_inANDdecr_above_zero


[]

In [75]:
w3= list()
w4= list()
i2=0

if len(inANDdecr_below_zero_and_inANDdecr_above_zero)>0:
    
    while i2 < len(inANDdecr_below_zero_and_inANDdecr_above_zero):
    
        i= 0
        r_id = inANDdecr_below_zero_and_inANDdecr_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        l= list()

        while i < int(((len(csvFile)-1)/2)+1):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and (l[n2+3] < max(l) or l[n2+3] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

    
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and (l[n2-3] < max(l) or l[n2-3] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
            
        i2= i2+1

w3


[]

In [76]:
union = pd.Series(np.union1d(w3, w4))
unclassified_below_zero_and_inANDdecr_above_zero2 = pd.Series(np.intersect1d(w3, w4))

set1= set(w3)
set2= set(unclassified_below_zero_and_inANDdecr_above_zero2)
set3= set1-set2
w3= list(set3)

set1= set(w4)
set2= set(unclassified_below_zero_and_inANDdecr_above_zero2)
set3= set1-set2
w4= list(set3)

unclassified_below_zero_and_inANDdecr_above_zero= list(unclassified_below_zero_and_inANDdecr_above_zero)+list(unclassified_below_zero_and_inANDdecr_above_zero2)
unclassified_below_zero_and_inANDdecr_above_zero


[]

In [77]:
w4

set1= set(inANDdecr_below_zero_and_inANDdecr_above_zero)
set2= set(w3)
set3= set(w4)
set4= set(unclassified_below_zero_and_inANDdecr_above_zero)
set5= set1-set2-set3-set4
inANDdecr_below_zero_and_inANDdecr_above_zero= list(set5)

mon_decr_below_zero_and_inANDdecr_above_zero= list(mon_decr_below_zero_and_inANDdecr_above_zero) + list(w3)
mon_incr_below_zero_and_inANDdecr_above_zero= list(mon_incr_below_zero_and_inANDdecr_above_zero) + list(w4)


In [78]:
i2= 0
mon_decr_below_zero_and_indecr_and_stable_above_zero= list()
mon_incr_below_zero_and_indecr_and_stable_above_zero= list()

if len(diff_indecr_below_zero_and_indecr_and_stable_above_zero)>0:
    
    for r_id in diff_indecr_below_zero_and_indecr_and_stable_above_zero:
    
        r_id = diff_indecr_below_zero_and_indecr_and_stable_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        n2= 0
        l= list()
    
        while n2 < int((len(csvFile)-1)/2+1):
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                mon_decr_below_zero_and_indecr_and_stable_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                mon_incr_below_zero_and_indecr_and_stable_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(diff_indecr_below_zero_and_indecr_and_stable_above_zero)
set2= set(mon_decr_below_zero_and_indecr_and_stable_above_zero)
set3= set(mon_incr_below_zero_and_indecr_and_stable_above_zero)
inANDdecr_below_zero_and_indecr_and_stable_above_zero= list(set1-set2-set3)
inANDdecr_below_zero_and_indecr_and_stable_above_zero


[]

In [79]:
w3= list()
w4= list()
i2=0

if len(inANDdecr_below_zero_and_indecr_and_stable_above_zero)>0:
    
    while i2 < len(inANDdecr_below_zero_and_indecr_and_stable_above_zero):
    
        i= 0
        r_id = inANDdecr_below_zero_and_indecr_and_stable_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        l= list()

        while i < int(((len(csvFile)-1)/2)+1):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and (l[n2+3] < max(l) or l[n2+3] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

    
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and (l[n2-3] < max(l) or l[n2-3] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
            
        i2= i2+1

w3


[]

In [80]:
union = pd.Series(np.union1d(w3, w4))
unclassified_below_zero_and_indecr_and_stable_above_zero = pd.Series(np.intersect1d(w3, w4))

set1= set(w3)
set2= set(unclassified_below_zero_and_indecr_and_stable_above_zero)
set3= set1-set2
w3= list(set3)

set1= set(w4)
set2= set(unclassified_below_zero_and_indecr_and_stable_above_zero)
set3= set1-set2
w4= list(set3)

unclassified_below_zero_and_indecr_and_stable_above_zero= list(unclassified_below_zero_and_indecr_and_stable_above_zero)
unclassified_below_zero_and_indecr_and_stable_above_zero


[]

In [81]:
w4

set1= set(inANDdecr_below_zero_and_indecr_and_stable_above_zero)
set2= set(w3)
set3= set(w4)
set4= set(unclassified_below_zero_and_indecr_and_stable_above_zero)
set5= set1-set2-set3-set4
inANDdecr_below_zero_and_indecr_and_stable_above_zero= list(set5)

mon_decr_below_zero_and_indecr_and_stable_above_zero= list(mon_decr_below_zero_and_indecr_and_stable_above_zero) + list(w3)
mon_incr_below_zero_and_indecr_and_stable_above_zero= list(mon_incr_below_zero_and_indecr_and_stable_above_zero) + list(w4)


In [82]:
i2= 0
inANDdecr_below_zero_and_mon_decr_and_stable_above_zero= list()
inANDdecr_below_zero_and_mon_incr_and_stable_above_zero= list()

if len(inANDdecr_below_zero_and_indecr_and_stable_above_zero)>0:
    
    for r_id in inANDdecr_below_zero_and_indecr_and_stable_above_zero:
    
        r_id = inANDdecr_below_zero_and_indecr_and_stable_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        n2= int((len(csvFile)-1)/2)
        l= list()
    
        while n2 < len(csvFile):
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                inANDdecr_below_zero_and_mon_decr_and_stable_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                inANDdecr_below_zero_and_mon_incr_and_stable_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(inANDdecr_below_zero_and_indecr_and_stable_above_zero)
set2= set(inANDdecr_below_zero_and_mon_decr_and_stable_above_zero)
set3= set(inANDdecr_below_zero_and_mon_incr_and_stable_above_zero)
inANDdecr_below_zero_and_inANDdecr_and_stable_above_zero= list(set1-set2-set3)
inANDdecr_below_zero_and_inANDdecr_and_stable_above_zero


[]

In [83]:
w3= list()
w4= list()
i2=0

if len(inANDdecr_below_zero_and_inANDdecr_and_stable_above_zero)>0:
    
    while i2 < len(inANDdecr_below_zero_and_inANDdecr_and_stable_above_zero):
    
        i= int((len(csvFile)-1)/2)
        r_id = inANDdecr_below_zero_and_inANDdecr_and_stable_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        l= list()

        while i < len(csvFile):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and (l[n2+3] < max(l) or l[n2+3] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

    
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and (l[n2-3] < max(l) or l[n2-3] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
            
        i2= i2+1

w3


[]

In [84]:
union = pd.Series(np.union1d(w3, w4))
inANDdecr_below_zero_and_unclassified_above_zero = pd.Series(np.intersect1d(w3, w4))

set1= set(w3)
set2= set(inANDdecr_below_zero_and_unclassified_above_zero)
set3= set1-set2
w3= list(set3)

set1= set(w4)
set2= set(inANDdecr_below_zero_and_unclassified_above_zero)
set3= set1-set2
w4= list(set3)

inANDdecr_below_zero_and_unclassified_above_zero= list(inANDdecr_below_zero_and_unclassified_above_zero)
inANDdecr_below_zero_and_unclassified_above_zero


[]

In [85]:
w4

set1= set(inANDdecr_below_zero_and_inANDdecr_and_stable_above_zero)
set2= set(w3)
set3= set(w4)
set4= set(inANDdecr_below_zero_and_unclassified_above_zero)
set5= set1-set2-set3-set4
inANDdecr_below_zero_and_inANDdecr_and_stable_above_zero= list(set5)

inANDdecr_below_zero_and_mon_decr_and_stable_above_zero= list(inANDdecr_below_zero_and_mon_decr_and_stable_above_zero) + list(w3)
inANDdecr_below_zero_and_mon_incr_and_stable_above_zero= list(inANDdecr_below_zero_and_mon_incr_and_stable_above_zero) + list(w4)


In [86]:
i2= 0
mon_decr_below_zero_and_mon_decr_and_stable_above_zero= list()
mon_decr_below_zero_and_mon_incr_and_stable_above_zero= list()

if len(mon_decr_below_zero_and_indecr_and_stable_above_zero)>0:
    
    for r_id in mon_decr_below_zero_and_indecr_and_stable_above_zero:
    
        r_id = mon_decr_below_zero_and_indecr_and_stable_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        n2= int((len(csvFile)-1)/2)
        l= list()
    
        while n2 < len(csvFile):
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                mon_decr_below_zero_and_mon_decr_and_stable_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                mon_decr_below_zero_and_mon_incr_and_stable_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(mon_decr_below_zero_and_indecr_and_stable_above_zero)
set2= set(mon_decr_below_zero_and_mon_decr_and_stable_above_zero)
set3= set(mon_decr_below_zero_and_mon_incr_and_stable_above_zero)
mon_decr_below_zero_and_inANDdecr_and_stable_above_zero= list(set1-set2-set3)
mon_decr_below_zero_and_inANDdecr_and_stable_above_zero


[]

In [87]:
w3= list()
w4= list()
i2=0

if len(mon_decr_below_zero_and_inANDdecr_and_stable_above_zero)>0:
    
    while i2 < len(mon_decr_below_zero_and_inANDdecr_and_stable_above_zero):
    
        i= int((len(csvFile)-1)/2)
        r_id = mon_decr_below_zero_and_inANDdecr_and_stable_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        l= list()

        while i < len(csvFile):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and (l[n2+3] < max(l) or l[n2+3] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

    
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and (l[n2-3] < max(l) or l[n2-3] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
            
        i2= i2+1

w3


[]

In [88]:
union = pd.Series(np.union1d(w3, w4))
mon_decr_below_zero_and_unclassified_above_zero = pd.Series(np.intersect1d(w3, w4))

set1= set(w3)
set2= set(mon_decr_below_zero_and_unclassified_above_zero)
set3= set1-set2
w3= list(set3)

set1= set(w4)
set2= set(mon_decr_below_zero_and_unclassified_above_zero)
set3= set1-set2
w4= list(set3)

mon_decr_below_zero_and_unclassified_above_zero= list(mon_decr_below_zero_and_unclassified_above_zero)
mon_decr_below_zero_and_unclassified_above_zero


[]

In [89]:
w4

set1= set(mon_decr_below_zero_and_inANDdecr_and_stable_above_zero)
set2= set(w3)
set3= set(w4)
set4= set(mon_decr_below_zero_and_unclassified_above_zero)
set5= set1-set2-set3-set4
mon_decr_below_zero_and_inANDdecr_and_stable_above_zero= list(set5)

mon_decr_below_zero_and_mon_decr_and_stable_above_zero= list(mon_decr_below_zero_and_mon_decr_and_stable_above_zero) + list(w3)
mon_decr_below_zero_and_mon_incr_and_stable_above_zero= list(mon_decr_below_zero_and_mon_incr_and_stable_above_zero) + list(w4)


In [90]:
i2= 0
mon_incr_below_zero_and_mon_decr_and_stable_above_zero= list()
mon_incr_below_zero_and_mon_incr_and_stable_above_zero= list()

if len(mon_incr_below_zero_and_indecr_and_stable_above_zero)>0:
    
    for r_id in mon_incr_below_zero_and_indecr_and_stable_above_zero:
    
        r_id = mon_incr_below_zero_and_indecr_and_stable_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        n2= int((len(csvFile)-1)/2)
        l= list()
    
        while n2 < len(csvFile):
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                mon_incr_below_zero_and_mon_decr_and_stable_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                mon_incr_below_zero_and_mon_incr_and_stable_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(mon_incr_below_zero_and_indecr_and_stable_above_zero)
set2= set(mon_incr_below_zero_and_mon_decr_and_stable_above_zero)
set3= set(mon_incr_below_zero_and_mon_incr_and_stable_above_zero)
mon_incr_below_zero_and_inANDdecr_and_stable_above_zero= list(set1-set2-set3)
mon_incr_below_zero_and_inANDdecr_and_stable_above_zero


['ENO',
 'MDH',
 'ECOAH1',
 'FUM',
 'ECOAH3',
 'MTHFD',
 'ECOAH2',
 'TPI',
 'GAPD',
 'MTHFC']

In [91]:
w3= list()
w4= list()
i2=0

if len(mon_incr_below_zero_and_inANDdecr_and_stable_above_zero)>0:
    
    while i2 < len(mon_incr_below_zero_and_inANDdecr_and_stable_above_zero):
    
        i= int((len(csvFile)-1)/2)
        r_id = mon_incr_below_zero_and_inANDdecr_and_stable_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        l= list()

        while i < len(csvFile):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and (l[n2+3] < max(l) or l[n2+3] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

    
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and (l[n2-3] < max(l) or l[n2-3] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
            
        i2= i2+1

w3


['ECOAH1', 'ECOAH3', 'ECOAH2']

In [92]:
union = pd.Series(np.union1d(w3, w4))
mon_incr_below_zero_and_unclassified_above_zero = pd.Series(np.intersect1d(w3, w4))

set1= set(w3)
set2= set(mon_incr_below_zero_and_unclassified_above_zero)
set3= set1-set2
w3= list(set3)

set1= set(w4)
set2= set(mon_incr_below_zero_and_unclassified_above_zero)
set3= set1-set2
w4= list(set3)

mon_incr_below_zero_and_unclassified_above_zero= list(mon_incr_below_zero_and_unclassified_above_zero)
mon_incr_below_zero_and_unclassified_above_zero


[]

In [93]:
w4

set1= set(mon_incr_below_zero_and_inANDdecr_and_stable_above_zero)
set2= set(w3)
set3= set(w4)
set4= set(mon_incr_below_zero_and_unclassified_above_zero)
set5= set1-set2-set3-set4
mon_incr_below_zero_and_inANDdecr_and_stable_above_zero= list(set5)

mon_incr_below_zero_and_mon_decr_and_stable_above_zero= list(mon_incr_below_zero_and_mon_decr_and_stable_above_zero) + list(w3)
mon_incr_below_zero_and_mon_incr_and_stable_above_zero= list(mon_incr_below_zero_and_mon_incr_and_stable_above_zero) + list(w4)


In [94]:
i2= 0
mon_decr_below_zero_and_one_to_three_data_point_above_zero= list()
mon_incr_below_zero_and_one_to_three_data_point_above_zero= list()

if len(diff_indecr_below_zero_and_one_to_three_data_point_above_zero)>0:
    
    for r_id in diff_indecr_below_zero_and_one_to_three_data_point_above_zero:
    
        r_id = diff_indecr_below_zero_and_one_to_three_data_point_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        n2= 0
        l= list()
    
        while n2 < int((len(csvFile)-1)/2+1):
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                mon_decr_below_zero_and_one_to_three_data_point_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                mon_incr_below_zero_and_one_to_three_data_point_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(diff_indecr_below_zero_and_one_to_three_data_point_above_zero)
set2= set(mon_decr_below_zero_and_one_to_three_data_point_above_zero)
set3= set(mon_incr_below_zero_and_one_to_three_data_point_above_zero)
inANDdecr_below_zero_and_one_to_three_data_point_above_zero= list(set1-set2-set3)
inANDdecr_below_zero_and_one_to_three_data_point_above_zero


[]

In [95]:
w3= list()
w4= list()
i2=0

if len(inANDdecr_below_zero_and_one_to_three_data_point_above_zero)>0:
    
    while i2 < len(inANDdecr_below_zero_and_one_to_three_data_point_above_zero):
    
        i= 0
        r_id = inANDdecr_below_zero_and_one_to_three_data_point_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        l= list()

        while i < int((len(csvFile)-1)/2+1):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and (l[n2+3] < max(l) or l[n2+3] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

    
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and (l[n2-3] < max(l) or l[n2-3] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
            
        i2= i2+1

w3


[]

In [96]:
union = pd.Series(np.union1d(w3, w4))
unclassified_below_zero_and_one_to_three_data_point_above_zero2 = pd.Series(np.intersect1d(w3, w4))

set1= set(w3)
set2= set(unclassified_below_zero_and_one_to_three_data_point_above_zero2)
set3= set1-set2
w3= list(set3)

set1= set(w4)
set2= set(unclassified_below_zero_and_one_to_three_data_point_above_zero2)
set3= set1-set2
w4= list(set3)

unclassified_below_zero_and_one_to_three_data_point_above_zero= list(unclassified_below_zero_and_one_to_three_data_point_above_zero)+list(unclassified_below_zero_and_one_to_three_data_point_above_zero2)
unclassified_below_zero_and_one_to_three_data_point_above_zero


[]

In [97]:
w4

set1= set(inANDdecr_below_zero_and_one_to_three_data_point_above_zero)
set2= set(w3)
set3= set(w4)
set4= set(unclassified_below_zero_and_one_to_three_data_point_above_zero)
set5= set1-set2-set3-set4
inANDdecr_below_zero_and_one_to_three_data_point_above_zero= list(set5)

mon_decr_below_zero_and_one_to_three_data_point_above_zero= list(mon_decr_below_zero_and_one_to_three_data_point_above_zero) + list(w3)
mon_incr_below_zero_and_one_to_three_data_point_above_zero= list(mon_incr_below_zero_and_one_to_three_data_point_above_zero) + list(w4)


In [98]:
i2= 0
flat_below_zero_and_mon_decr_above_zero= list()
flat_below_zero_and_mon_incr_above_zero= list()

if len(flat_below_zero_and_diff_indecr_above_zero)>0:
    
    for r_id in flat_below_zero_and_diff_indecr_above_zero:
    
        r_id = flat_below_zero_and_diff_indecr_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        n2= int((len(csvFile)-1)/2)
        l= list()
    
        while n2 < len(csvFile):
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                flat_below_zero_and_mon_decr_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                flat_below_zero_and_mon_incr_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(flat_below_zero_and_diff_indecr_above_zero)
set2= set(flat_below_zero_and_mon_decr_above_zero)
set3= set(flat_below_zero_and_mon_incr_above_zero)
flat_below_zero_and_inANDdecr_above_zero= list(set1-set2-set3)
flat_below_zero_and_inANDdecr_above_zero


[]

In [99]:
w3= list()
w4= list()
i2=0

if len(flat_below_zero_and_inANDdecr_above_zero)>0:
    
    while i2 < len(flat_below_zero_and_inANDdecr_above_zero):
    
        i= int((len(csvFile)-1)/2)
        r_id = flat_below_zero_and_inANDdecr_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        l= list()

        while i < len(csvFile):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and (l[n2+3] < max(l) or l[n2+3] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

    
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and (l[n2-3] < max(l) or l[n2-3] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
            
        i2= i2+1

w3


[]

In [100]:
union = pd.Series(np.union1d(w3, w4))
flat_below_zero_and_unclassified_above_zero = pd.Series(np.intersect1d(w3, w4))

set1= set(w3)
set2= set(flat_below_zero_and_unclassified_above_zero)
set3= set1-set2
w3= list(set3)

set1= set(w4)
set2= set(flat_below_zero_and_unclassified_above_zero)
set3= set1-set2
w4= list(set3)

flat_below_zero_and_unclassified_above_zero= list(flat_below_zero_and_unclassified_above_zero)
flat_below_zero_and_unclassified_above_zero


[]

In [101]:
w4

set1= set(flat_below_zero_and_inANDdecr_above_zero)
set2= set(w3)
set3= set(w4)
set4= set(flat_below_zero_and_unclassified_above_zero)
set5= set1-set2-set3-set4
flat_below_zero_and_inANDdecr_above_zero= list(set5)

flat_below_zero_and_mon_decr_above_zero= list(flat_below_zero_and_mon_decr_above_zero) + list(w3)
flat_below_zero_and_mon_incr_above_zero= list(flat_below_zero_and_mon_incr_above_zero) + list(w4)


In [102]:
i2= 0
flat_below_zero_and_mon_decr_and_stable_above_zero= list()
flat_below_zero_and_mon_incr_and_stable_above_zero= list()

if len(flat_below_zero_and_indecr_and_stable_above_zero)>0:
    
    for r_id in flat_below_zero_and_indecr_and_stable_above_zero:
    
        r_id = flat_below_zero_and_indecr_and_stable_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        n2= int((len(csvFile)-1)/2)
        l= list()
    
        while n2 < len(csvFile):
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                flat_below_zero_and_mon_decr_and_stable_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                flat_below_zero_and_mon_incr_and_stable_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(flat_below_zero_and_indecr_and_stable_above_zero)
set2= set(flat_below_zero_and_mon_decr_and_stable_above_zero)
set3= set(flat_below_zero_and_mon_incr_and_stable_above_zero)
flat_below_zero_and_inANDdecr_and_stable_above_zero= list(set1-set2-set3)
flat_below_zero_and_inANDdecr_and_stable_above_zero


[]

In [103]:
w3= list()
w4= list()
i2=0

if len(flat_below_zero_and_inANDdecr_and_stable_above_zero)>0:
    
    while i2 < len(flat_below_zero_and_inANDdecr_and_stable_above_zero):
    
        i= int((len(csvFile)-1)/2)
        r_id = flat_below_zero_and_inANDdecr_and_stable_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        l= list()

        while i < len(csvFile):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and (l[n2+3] < max(l) or l[n2+3] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

    
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and (l[n2-3] < max(l) or l[n2-3] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
            
        i2= i2+1

w3


[]

In [104]:
union = pd.Series(np.union1d(w3, w4))
flat_below_zero_and_unclassified_above_zero2 = pd.Series(np.intersect1d(w3, w4))

set1= set(w3)
set2= set(flat_below_zero_and_unclassified_above_zero2)
set3= set1-set2
w3= list(set3)

set1= set(w4)
set2= set(flat_below_zero_and_unclassified_above_zero2)
set3= set1-set2
w4= list(set3)

flat_below_zero_and_unclassified_above_zero= list(flat_below_zero_and_unclassified_above_zero)+list(flat_below_zero_and_unclassified_above_zero2)
flat_below_zero_and_unclassified_above_zero


[]

In [105]:
w4

set1= set(flat_below_zero_and_inANDdecr_and_stable_above_zero)
set2= set(w3)
set3= set(w4)
set4= set(flat_below_zero_and_unclassified_above_zero)
set5= set1-set2-set3-set4
flat_below_zero_and_inANDdecr_and_stable_above_zero= list(set5)

flat_below_zero_and_mon_decr_and_stable_above_zero= list(flat_below_zero_and_mon_decr_and_stable_above_zero) + list(w3)
flat_below_zero_and_mon_incr_and_stable_above_zero= list(flat_below_zero_and_mon_incr_and_stable_above_zero) + list(w4)


In [106]:
i2= 0
mon_decr_below_zero_and_flat_above_zero= list()
mon_incr_below_zero_and_flat_above_zero= list()

if len(diff_indecr_below_zero_and_flat_above_zero)>0:
    
    for r_id in diff_indecr_below_zero_and_flat_above_zero:
    
        r_id = diff_indecr_below_zero_and_flat_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        n2= 0
        l= list()
    
        while n2 < int((len(csvFile)-1)/2+1):
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                mon_decr_below_zero_and_flat_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                mon_incr_below_zero_and_flat_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(diff_indecr_below_zero_and_flat_above_zero)
set2= set(mon_decr_below_zero_and_flat_above_zero)
set3= set(mon_incr_below_zero_and_flat_above_zero)
inANDdecr_below_zero_and_flat_above_zero= list(set1-set2-set3)
inANDdecr_below_zero_and_flat_above_zero


[]

In [107]:
w3= list()
w4= list()
i2=0

if len(inANDdecr_below_zero_and_flat_above_zero)>0:
    
    while i2 < len(inANDdecr_below_zero_and_flat_above_zero):
    
        i= 0
        r_id = inANDdecr_below_zero_and_flat_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        l= list()

        while i < int((len(csvFile)-1)/2+1):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and (l[n2+3] < max(l) or l[n2+3] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

    
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and (l[n2-3] < max(l) or l[n2-3] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
        i2= i2+1

w3


[]

In [108]:
union = pd.Series(np.union1d(w3, w4))
unclassified_below_zero_and_flat_above_zero = pd.Series(np.intersect1d(w3, w4))

set1= set(w3)
set2= set(unclassified_below_zero_and_flat_above_zero)
set3= set1-set2
w3= list(set3)

set1= set(w4)
set2= set(unclassified_below_zero_and_flat_above_zero)
set3= set1-set2
w4= list(set3)

unclassified_below_zero_and_flat_above_zero= list(unclassified_below_zero_and_flat_above_zero)
unclassified_below_zero_and_flat_above_zero


[]

In [109]:
w4

set1= set(inANDdecr_below_zero_and_flat_above_zero)
set2= set(w3)
set3= set(w4)
set4= set(unclassified_below_zero_and_flat_above_zero)
set5= set1-set2-set3-set4
inANDdecr_below_zero_and_flat_above_zero= list(set5)

mon_decr_below_zero_and_flat_above_zero= list(mon_decr_below_zero_and_flat_above_zero) + list(w3)
mon_incr_below_zero_and_flat_above_zero= list(mon_incr_below_zero_and_flat_above_zero) + list(w4)


In [110]:
i2= 0
mon_decr_and_stable_below_zero_and_flat_above_zero= list()
mon_incr_and_stable_below_zero_and_flat_above_zero= list()

if len(indecr_and_stable_below_zero_and_flat_above_zero)>0:
    
    for r_id in indecr_and_stable_below_zero_and_flat_above_zero:
    
        r_id = indecr_and_stable_below_zero_and_flat_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        n2= 0
        l= list()
    
        while n2 < int((len(csvFile)-1)/2+1):
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                mon_decr_and_stable_below_zero_and_flat_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                mon_incr_and_stable_below_zero_and_flat_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(indecr_and_stable_below_zero_and_flat_above_zero)
set2= set(mon_decr_and_stable_below_zero_and_flat_above_zero)
set3= set(mon_incr_and_stable_below_zero_and_flat_above_zero)
inANDdecr_and_stable_below_zero_and_flat_above_zero= list(set1-set2-set3)
inANDdecr_and_stable_below_zero_and_flat_above_zero


[]

In [111]:
w3= list()
w4= list()
i2=0

if len(inANDdecr_and_stable_below_zero_and_flat_above_zero)>0:
    
    while i2 < len(inANDdecr_and_stable_below_zero_and_flat_above_zero):
    
        i= 0
        r_id = inANDdecr_and_stable_below_zero_and_flat_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        l= list()

        while i < int((len(csvFile)-1)/2+1):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and (l[n2+3] < max(l) or l[n2+3] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

    
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and (l[n2-3] < max(l) or l[n2-3] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
            
        i2= i2+1

w3


[]

In [112]:
union = pd.Series(np.union1d(w3, w4))
unclassified_below_zero_and_flat_above_zero2 = pd.Series(np.intersect1d(w3, w4))

set1= set(w3)
set2= set(unclassified_below_zero_and_flat_above_zero2)
set3= set1-set2
w3= list(set3)

set1= set(w4)
set2= set(unclassified_below_zero_and_flat_above_zero2)
set3= set1-set2
w4= list(set3)

unclassified_below_zero_and_flat_above_zero= list(unclassified_below_zero_and_flat_above_zero)+list(unclassified_below_zero_and_flat_above_zero2)
unclassified_below_zero_and_flat_above_zero


[]

In [113]:
w4

set1= set(inANDdecr_and_stable_below_zero_and_flat_above_zero)
set2= set(w3)
set3= set(w4)
set4= set(unclassified_below_zero_and_flat_above_zero)
set5= set1-set2-set3-set4
inANDdecr_and_stable_below_zero_and_flat_above_zero= list(set5)

mon_decr_and_stable_below_zero_and_flat_above_zero= list(mon_decr_and_stable_below_zero_and_flat_above_zero) + list(w3)
mon_incr_and_stable_below_zero_and_flat_above_zero= list(mon_incr_and_stable_below_zero_and_flat_above_zero) + list(w4)


In [114]:
i2= 0
mon_decr_below_zero_and_flat_above_zero= list()
mon_incr_below_zero_and_flat_above_zero= list()

if len(diff_indecr_below_zero_and_flat_above_zero)>0:
    
    for r_id in diff_indecr_below_zero_and_flat_above_zero:
    
        r_id = diff_indecr_below_zero_and_flat_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        n2= 0
        l= list()
    
        while n2 < int((len(csvFile)-1)/2+1):
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                mon_decr_below_zero_and_flat_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                mon_incr_below_zero_and_flat_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(diff_indecr_below_zero_and_flat_above_zero)
set2= set(mon_decr_below_zero_and_flat_above_zero)
set3= set(mon_incr_below_zero_and_flat_above_zero)
inANDdecr_below_zero_and_flat_above_zero= list(set1-set2-set3)
inANDdecr_below_zero_and_flat_above_zero


[]

In [115]:
w3= list()
w4= list()
i2=0

if len(inANDdecr_below_zero_and_flat_above_zero)>0:
    
    while i2 < len(inANDdecr_below_zero_and_flat_above_zero):
    
        i= 0
        r_id = inANDdecr_below_zero_and_flat_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        l= list()

        while i < int((len(csvFile)-1)/2+1):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and (l[n2+3] < max(l) or l[n2+3] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

    
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and (l[n2-3] < max(l) or l[n2-3] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
            
        i2= i2+1

w3


[]

In [116]:
union = pd.Series(np.union1d(w3, w4))
unclassified_below_zero_and_flat_above_zero2 = pd.Series(np.intersect1d(w3, w4))

set1= set(w3)
set2= set(unclassified_below_zero_and_flat_above_zero2)
set3= set1-set2
w3= list(set3)

set1= set(w4)
set2= set(unclassified_below_zero_and_flat_above_zero2)
set3= set1-set2
w4= list(set3)

unclassified_below_zero_and_flat_above_zero= list(unclassified_below_zero_and_flat_above_zero)+list(unclassified_below_zero_and_flat_above_zero2)
unclassified_below_zero_and_flat_above_zero


[]

In [117]:
w4

set1= set(inANDdecr_below_zero_and_flat_above_zero)
set2= set(w3)
set3= set(w4)
set4= set(unclassified_below_zero_and_flat_above_zero)
set5= set1-set2-set3-set4
inANDdecr_below_zero_and_flat_above_zero= list(set5)

mon_decr_below_zero_and_flat_above_zero= list(mon_decr_below_zero_and_flat_above_zero) + list(w3)
mon_incr_below_zero_and_flat_above_zero= list(mon_incr_below_zero_and_flat_above_zero) + list(w4)


In [118]:
i2= 0
mon_decr_and_stable_below_zero_and_indecr_and_stable_above_zero= list()
mon_incr_and_stable_below_zero_and_indecr_and_stable_above_zero= list()

if len(indecr_and_stable_below_zero_and_indecr_and_stable_above_zero)>0:
    
    for r_id in indecr_and_stable_below_zero_and_indecr_and_stable_above_zero:
    
        r_id = indecr_and_stable_below_zero_and_indecr_and_stable_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        n2= 0
        l= list()
    
        while n2 < int((len(csvFile)-1)/2+1):
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                mon_decr_and_stable_below_zero_and_indecr_and_stable_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                mon_incr_and_stable_below_zero_and_indecr_and_stable_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(indecr_and_stable_below_zero_and_indecr_and_stable_above_zero)
set2= set(mon_decr_and_stable_below_zero_and_indecr_and_stable_above_zero)
set3= set(mon_incr_and_stable_below_zero_and_indecr_and_stable_above_zero)
inANDdecr_and_stable_below_zero_and_indecr_and_stable_above_zero= list(set1-set2-set3)
inANDdecr_and_stable_below_zero_and_indecr_and_stable_above_zero


['FBP', 'CITL', 'FLDR2', 'PPCK', 'ME2', 'THD2pp', 'PPS']

In [119]:
w3= list()
w4= list()
i2=0

if len(inANDdecr_and_stable_below_zero_and_indecr_and_stable_above_zero)>0:
    
    while i2 < len(inANDdecr_and_stable_below_zero_and_indecr_and_stable_above_zero):
    
        i= 0
        r_id = inANDdecr_and_stable_below_zero_and_indecr_and_stable_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        l= list()

        while i < int((len(csvFile)-1)/2+1):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and (l[n2+3] < max(l) or l[n2+3] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

    
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and (l[n2-3] < max(l) or l[n2-3] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
            
        i2= i2+1

w3


[]

In [120]:
union = pd.Series(np.union1d(w3, w4))
unclassified_below_zero_and_indecr_and_stable_above_zero2 = pd.Series(np.intersect1d(w3, w4))

set1= set(w3)
set2= set(unclassified_below_zero_and_indecr_and_stable_above_zero2)
set3= set1-set2
w3= list(set3)

set1= set(w4)
set2= set(unclassified_below_zero_and_indecr_and_stable_above_zero2)
set3= set1-set2
w4= list(set3)

unclassified_below_zero_and_indecr_and_stable_above_zero= list(unclassified_below_zero_and_indecr_and_stable_above_zero)+list(unclassified_below_zero_and_indecr_and_stable_above_zero2)
unclassified_below_zero_and_indecr_and_stable_above_zero


[]

In [121]:
w4

set1= set(inANDdecr_and_stable_below_zero_and_indecr_and_stable_above_zero)
set2= set(w3)
set3= set(w4)
set4= set(unclassified_below_zero_and_indecr_and_stable_above_zero)
set5= set1-set2-set3-set4
inANDdecr_and_stable_below_zero_and_indecr_and_stable_above_zero= list(set5)

mon_decr_and_stable_below_zero_and_indecr_and_stable_above_zero= list(mon_decr_and_stable_below_zero_and_indecr_and_stable_above_zero) + list(w3)
mon_incr_and_stable_below_zero_and_indecr_and_stable_above_zero= list(mon_incr_and_stable_below_zero_and_indecr_and_stable_above_zero) + list(w4)


In [122]:
i2= 0
inANDdecr_and_stable_below_zero_and_mon_decr_and_stable_above_zero= list()
inANDdecr_and_stable_below_zero_and_mon_incr_and_stable_above_zero= list()

if len(inANDdecr_and_stable_below_zero_and_indecr_and_stable_above_zero)>0:
    
    for r_id in inANDdecr_and_stable_below_zero_and_indecr_and_stable_above_zero:
    
        r_id = inANDdecr_and_stable_below_zero_and_indecr_and_stable_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        n2= int((len(csvFile)-1)/2)
        l= list()
    
        while n2 < len(csvFile):
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                inANDdecr_and_stable_below_zero_and_mon_decr_and_stable_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                inANDdecr_and_stable_below_zero_and_mon_incr_and_stable_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(inANDdecr_and_stable_below_zero_and_indecr_and_stable_above_zero)
set2= set(inANDdecr_and_stable_below_zero_and_mon_decr_and_stable_above_zero)
set3= set(inANDdecr_and_stable_below_zero_and_mon_incr_and_stable_above_zero)
inANDdecr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero= list(set1-set2-set3)
inANDdecr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero


[]

In [123]:
w3= list()
w4= list()
i2=0

if len(inANDdecr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero)>0:
    
    while i2 < len(inANDdecr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero):
    
        i= int((len(csvFile)-1)/2)
        r_id = inANDdecr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        l= list()

        while i < len(csvFile):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and (l[n2+3] < max(l) or l[n2+3] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

    
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and (l[n2-3] < max(l) or l[n2-3] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
            
        i2= i2+1

w3


[]

In [124]:
union = pd.Series(np.union1d(w3, w4))
inANDdecr_and_stable_below_zero_and_unclassified_above_zero = pd.Series(np.intersect1d(w3, w4))

set1= set(w3)
set2= set(inANDdecr_and_stable_below_zero_and_unclassified_above_zero)
set3= set1-set2
w3= list(set3)

set1= set(w4)
set2= set(inANDdecr_and_stable_below_zero_and_unclassified_above_zero)
set3= set1-set2
w4= list(set3)

inANDdecr_and_stable_below_zero_and_unclassified_above_zero= list(inANDdecr_and_stable_below_zero_and_unclassified_above_zero)
inANDdecr_and_stable_below_zero_and_unclassified_above_zero


[]

In [125]:
w4

set1= set(inANDdecr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero)
set2= set(w3)
set3= set(w4)
set4= set(inANDdecr_and_stable_below_zero_and_unclassified_above_zero)
set5= set1-set2-set3-set4
inANDdecr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero= list(set5)

inANDdecr_and_stable_below_zero_and_mon_decr_and_stable_above_zero= list(inANDdecr_and_stable_below_zero_and_mon_decr_and_stable_above_zero) + list(w3)
inANDdecr_and_stable_below_zero_and_mon_incr_and_stable_above_zero= list(inANDdecr_and_stable_below_zero_and_mon_incr_and_stable_above_zero) + list(w4)


In [126]:
i2= 0
mon_decr_and_stable_below_zero_and_mon_decr_and_stable_above_zero= list()
mon_decr_and_stable_below_zero_and_mon_incr_and_stable_above_zero= list()

if len(mon_decr_and_stable_below_zero_and_indecr_and_stable_above_zero)>0:
    
    for r_id in mon_decr_and_stable_below_zero_and_indecr_and_stable_above_zero:
    
        r_id = mon_decr_and_stable_below_zero_and_indecr_and_stable_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        n2= int((len(csvFile)-1)/2)
        l= list()
    
        while n2 < len(csvFile):
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                mon_decr_and_stable_below_zero_and_mon_decr_and_stable_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                mon_decr_and_stable_below_zero_and_mon_incr_and_stable_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(mon_decr_and_stable_below_zero_and_indecr_and_stable_above_zero)
set2= set(mon_decr_and_stable_below_zero_and_mon_decr_and_stable_above_zero)
set3= set(mon_decr_and_stable_below_zero_and_mon_incr_and_stable_above_zero)
mon_decr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero= list(set1-set2-set3)
mon_decr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero


[]

In [127]:
w3= list()
w4= list()
i2=0

if len(mon_decr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero)>0:
    
    while i2 < len(mon_decr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero):
    
        i= int((len(csvFile)-1)/2)
        r_id = mon_decr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        l= list()

        while i < len(csvFile):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and (l[n2+3] < max(l) or l[n2+3] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

    
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and (l[n2-3] < max(l) or l[n2-3] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
            
        i2= i2+1

w3


[]

In [128]:
union = pd.Series(np.union1d(w3, w4))
mon_decr_and_stable_below_zero_and_unclassified_above_zero = pd.Series(np.intersect1d(w3, w4))

set1= set(w3)
set2= set(mon_decr_and_stable_below_zero_and_unclassified_above_zero)
set3= set1-set2
w3= list(set3)

set1= set(w4)
set2= set(mon_decr_and_stable_below_zero_and_unclassified_above_zero)
set3= set1-set2
w4= list(set3)

mon_decr_and_stable_below_zero_and_unclassified_above_zero= list(mon_decr_and_stable_below_zero_and_unclassified_above_zero)
mon_decr_and_stable_below_zero_and_unclassified_above_zero


[]

In [129]:
w4

set1= set(mon_decr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero)
set2= set(w3)
set3= set(w4)
set4= set(mon_decr_and_stable_below_zero_and_unclassified_above_zero)
set5= set1-set2-set3-set4
mon_decr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero= list(set5)

mon_decr_and_stable_below_zero_and_mon_decr_and_stable_above_zero= list(mon_decr_and_stable_below_zero_and_mon_decr_and_stable_above_zero) + list(w3)
mon_decr_and_stable_below_zero_and_mon_incr_and_stable_above_zero= list(mon_decr_and_stable_below_zero_and_mon_incr_and_stable_above_zero) + list(w4)


In [130]:
i2= 0
mon_incr_and_stable_below_zero_and_mon_decr_and_stable_above_zero= list()
mon_incr_and_stable_below_zero_and_mon_incr_and_stable_above_zero= list()

if len(mon_incr_and_stable_below_zero_and_indecr_and_stable_above_zero)>0:
    
    for r_id in mon_incr_and_stable_below_zero_and_indecr_and_stable_above_zero:
    
        r_id = mon_incr_and_stable_below_zero_and_indecr_and_stable_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        n2= int((len(csvFile)-1)/2)
        l= list()
    
        while n2 < len(csvFile):
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                mon_incr_and_stable_below_zero_and_mon_decr_and_stable_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                mon_incr_and_stable_below_zero_and_mon_incr_and_stable_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(mon_incr_and_stable_below_zero_and_indecr_and_stable_above_zero)
set2= set(mon_incr_and_stable_below_zero_and_mon_decr_and_stable_above_zero)
set3= set(mon_incr_and_stable_below_zero_and_mon_incr_and_stable_above_zero)
mon_incr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero= list(set1-set2-set3)
mon_incr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero


[]

In [131]:
w3= list()
w4= list()
i2=0

if len(mon_incr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero)>0:
    
    while i2 < len(mon_incr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero):
    
        i= int((len(csvFile)-1)/2)
        r_id = mon_incr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        l= list()

        while i < len(csvFile):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and (l[n2+3] < max(l) or l[n2+3] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

    
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and (l[n2-3] < max(l) or l[n2-3] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
            
        i2= i2+1

w3


[]

In [132]:
union = pd.Series(np.union1d(w3, w4))
mon_incr_and_stable_below_zero_and_unclassified_above_zero = pd.Series(np.intersect1d(w3, w4))

set1= set(w3)
set2= set(mon_incr_and_stable_below_zero_and_unclassified_above_zero)
set3= set1-set2
w3= list(set3)

set1= set(w4)
set2= set(mon_incr_and_stable_below_zero_and_unclassified_above_zero)
set3= set1-set2
w4= list(set3)

mon_incr_and_stable_below_zero_and_unclassified_above_zero= list(mon_incr_and_stable_below_zero_and_unclassified_above_zero)
mon_incr_and_stable_below_zero_and_unclassified_above_zero


[]

In [133]:
w4

set1= set(mon_incr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero)
set2= set(w3)
set3= set(w4)
set4= set(mon_incr_and_stable_below_zero_and_unclassified_above_zero)
set5= set1-set2-set3-set4
mon_incr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero= list(set5)

mon_incr_and_stable_below_zero_and_mon_decr_and_stable_above_zero= list(mon_incr_and_stable_below_zero_and_mon_decr_and_stable_above_zero) + list(w3)
mon_incr_and_stable_below_zero_and_mon_incr_and_stable_above_zero= list(mon_incr_and_stable_below_zero_and_mon_incr_and_stable_above_zero) + list(w4)


In [134]:
i2= 0
mon_decr_and_stable_below_zero_and_unclassified_above_zero= list()
mon_incr_and_stable_below_zero_and_unclassified_above_zero= list()

if len(indecr_and_stable_below_zero_and_unclassified_above_zero)>0:
    
    for r_id in indecr_and_stable_below_zero_and_unclassified_above_zero:
    
        r_id = indecr_and_stable_below_zero_and_unclassified_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        n2= 0
        l= list()
    
        while n2 < int((len(csvFile)-1)/2+1):
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                mon_decr_and_stable_below_zero_and_unclassified_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                mon_incr_and_stable_below_zero_and_unclassified_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(indecr_and_stable_below_zero_and_unclassified_above_zero)
set2= set(mon_decr_and_stable_below_zero_and_unclassified_above_zero)
set3= set(mon_incr_and_stable_below_zero_and_unclassified_above_zero)
inANDdecr_and_stable_below_zero_and_unclassified_above_zero= list(set1-set2-set3)
inANDdecr_and_stable_below_zero_and_unclassified_above_zero


[]

In [135]:
w3= list()
w4= list()
i2=0

if len(inANDdecr_and_stable_below_zero_and_unclassified_above_zero)>0:
    
    while i2 < len(inANDdecr_and_stable_below_zero_and_unclassified_above_zero):
    
        i= 0
        r_id = inANDdecr_and_stable_below_zero_and_unclassified_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        l= list()

        while i < int((len(csvFile)-1)/2+1):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and (l[n2+3] < max(l) or l[n2+3] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

    
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and (l[n2-3] < max(l) or l[n2-3] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
            
        i2= i2+1

w3


[]

In [136]:
union = pd.Series(np.union1d(w3, w4))
unclassified_below_zero_and_unclassified_above_zero = pd.Series(np.intersect1d(w3, w4))

set1= set(w3)
set2= set(unclassified_below_zero_and_unclassified_above_zero)
set3= set1-set2
w3= list(set3)

set1= set(w4)
set2= set(unclassified_below_zero_and_unclassified_above_zero)
set3= set1-set2
w4= list(set3)

unclassified_below_zero_and_unclassified_above_zero= list(unclassified_below_zero_and_unclassified_above_zero)
unclassified_below_zero_and_unclassified_above_zero


[]

In [137]:
w4

set1= set(inANDdecr_and_stable_below_zero_and_unclassified_above_zero)
set2= set(w3)
set3= set(w4)
set4= set(unclassified_below_zero_and_unclassified_above_zero)
set5= set1-set2-set3-set4
inANDdecr_and_stable_below_zero_and_unclassified_above_zero= list(set5)

mon_decr_and_stable_below_zero_and_unclassified_above_zero= list(mon_decr_and_stable_below_zero_and_unclassified_above_zero) + list(w3)
mon_incr_and_stable_below_zero_and_unclassified_above_zero= list(mon_incr_and_stable_below_zero_and_unclassified_above_zero) + list(w4)


In [138]:
i2= 0
unclassified_below_zero_and_mon_decr_and_stable_above_zero= list()
unclassified_below_zero_and_mon_incr_and_stable_above_zero= list()

if len(unclassified_below_zero_and_indecr_and_stable_above_zero)>0:
    
    for r_id in unclassified_below_zero_and_indecr_and_stable_above_zero:
    
        r_id = unclassified_below_zero_and_indecr_and_stable_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        n2= int((len(csvFile)-1)/2)
        l= list()
    
        while n2 < len(csvFile):
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                unclassified_below_zero_and_mon_decr_and_stable_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                unclassified_below_zero_and_mon_incr_and_stable_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(unclassified_below_zero_and_indecr_and_stable_above_zero)
set2= set(unclassified_below_zero_and_mon_decr_and_stable_above_zero)
set3= set(unclassified_below_zero_and_mon_incr_and_stable_above_zero)
unclassified_below_zero_and_inANDdecr_and_stable_above_zero= list(set1-set2-set3)
unclassified_below_zero_and_inANDdecr_and_stable_above_zero


[]

In [139]:
w3= list()
w4= list()
i2=0

if len(unclassified_below_zero_and_inANDdecr_and_stable_above_zero)>0:
    
    while i2 < len(unclassified_below_zero_and_inANDdecr_and_stable_above_zero):
    
        i= int((len(csvFile)-1)/2)
        r_id = unclassified_below_zero_and_inANDdecr_and_stable_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
        l= list()

        while i < len(csvFile):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and (l[n2+3] < max(l) or l[n2+3] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

    
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and (l[n2-3] < max(l) or l[n2-3] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
            
        i2= i2+1

w3


[]

In [140]:
union = pd.Series(np.union1d(w3, w4))
unclassified_below_zero_and_unclassified_above_zero2 = pd.Series(np.intersect1d(w3, w4))

set1= set(w3)
set2= set(unclassified_below_zero_and_unclassified_above_zero2)
set3= set1-set2
w3= list(set3)

set1= set(w4)
set2= set(unclassified_below_zero_and_unclassified_above_zero2)
set3= set1-set2
w4= list(set3)

unclassified_below_zero_and_unclassified_above_zero= list(unclassified_below_zero_and_unclassified_above_zero)+list(unclassified_below_zero_and_unclassified_above_zero2)
unclassified_below_zero_and_unclassified_above_zero


[]

In [141]:
w4

set1= set(unclassified_below_zero_and_inANDdecr_and_stable_above_zero)
set2= set(w3)
set3= set(w4)
set4= set(unclassified_below_zero_and_unclassified_above_zero)
set5= set1-set2-set3-set4
unclassified_below_zero_and_inANDdecr_and_stable_above_zero= list(set5)

unclassified_below_zero_and_mon_decr_and_stable_above_zero= list(unclassified_below_zero_and_mon_decr_and_stable_above_zero) + list(w3)
unclassified_below_zero_and_mon_incr_and_stable_above_zero= list(unclassified_below_zero_and_mon_incr_and_stable_above_zero) + list(w4)


In [142]:
one
zero
one_at_zero
flat_above_zero
flat_below_zero

mon_decr_and_stable_above_zero
mon_incr_and_stable_above_zero
inANDdecr_and_stable_above_zero

mon_decr_and_stable_below_zero
mon_incr_and_stable_below_zero
inANDdecr_and_stable_below_zero

decr_and_stable_below_zero_and_one_to_three_data_point_above_zero
incr_and_stable_below_zero_and_one_to_three_data_point_above_zero
inANDdecr_and_stable_below_zero_and_one_to_three_data_point_above_zero

decr_and_stable_below_zero_and_mon_decr_above_zero
incr_and_stable_below_zero_and_mon_decr_above_zero
inANDdecr_and_stable_below_zero_and_mon_decr_above_zero

decr_and_stable_below_zero_and_mon_incr_above_zero
incr_and_stable_below_zero_and_mon_incr_above_zero
inANDdecr_and_stable_below_zero_and_mon_incr_above_zero

decr_and_stable_below_zero_and_inANDdecr_above_zero
incr_and_stable_below_zero_and_inANDdecr_above_zero
inANDdecr_and_stable_below_zero_and_inANDdecr_above_zero

one_to_three_data_point_below_zero_and_decr_and_stable_above_zero
one_to_three_data_point_below_zero_and_incr_and_stable_above_zero
one_to_three_data_point_below_zero_and_inANDdecr_and_stable_above_zero

one_to_three_data_point_below_zero_and_mon_decr_above_zero
one_to_three_data_point_below_zero_and_mon_incr_above_zero
one_to_three_data_point_below_zero_and_inANDdecr_above_zero

inANDdecr_below_zero_and_mon_decr_above_zero
mon_decr_below_zero_and_mon_decr_above_zero
mon_incr_below_zero_and_mon_decr_above_zero

inANDdecr_below_zero_and_mon_incr_above_zero
mon_decr_below_zero_and_mon_incr_above_zero
mon_incr_below_zero_and_mon_incr_above_zero

inANDdecr_below_zero_and_inANDdecr_above_zero
mon_decr_below_zero_and_inANDdecr_above_zero
mon_incr_below_zero_and_inANDdecr_above_zero

inANDdecr_below_zero_and_mon_decr_and_stable_above_zero
inANDdecr_below_zero_and_mon_incr_and_stable_above_zero
inANDdecr_below_zero_and_inANDdecr_and_stable_above_zero

mon_decr_below_zero_and_inANDdecr_and_stable_above_zero
mon_decr_below_zero_and_mon_decr_and_stable_above_zero
mon_decr_below_zero_and_mon_incr_and_stable_above_zero

mon_incr_below_zero_and_inANDdecr_and_stable_above_zero
mon_incr_below_zero_and_mon_decr_and_stable_above_zero
mon_incr_below_zero_and_mon_incr_and_stable_above_zero

inANDdecr_below_zero_and_one_to_three_data_point_above_zero
mon_decr_below_zero_and_one_to_three_data_point_above_zero
mon_incr_below_zero_and_one_to_three_data_point_above_zero

flat_below_zero_and_inANDdecr_above_zero
flat_below_zero_and_mon_decr_above_zero
flat_below_zero_and_mon_incr_above_zero

flat_below_zero_and_inANDdecr_and_stable_above_zero
flat_below_zero_and_mon_decr_and_stable_above_zero
flat_below_zero_and_mon_incr_and_stable_above_zero

inANDdecr_and_stable_below_zero_and_flat_above_zero
mon_decr_and_stable_below_zero_and_flat_above_zero
mon_incr_and_stable_below_zero_and_flat_above_zero

inANDdecr_below_zero_and_flat_above_zero
mon_decr_below_zero_and_flat_above_zero
mon_incr_below_zero_and_flat_above_zero

inANDdecr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero
inANDdecr_and_stable_below_zero_and_mon_decr_and_stable_above_zero
inANDdecr_and_stable_below_zero_and_mon_incr_and_stable_above_zero

mon_decr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero
mon_decr_and_stable_below_zero_and_mon_decr_and_stable_above_zero
mon_decr_and_stable_below_zero_and_mon_incr_and_stable_above_zero

mon_incr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero
mon_incr_and_stable_below_zero_and_mon_decr_and_stable_above_zero
mon_incr_and_stable_below_zero_and_mon_incr_and_stable_above_zero

flat_below_zero_and_flat_above_zero
one_to_three_data_point_below_zero_and_flat_above_zero
one_to_three_data_point_below_zero_and_one_to_three_data_point_above_zero
flat_below_zero_and_one_to_three_data_point_above_zero

unclassified_above_zero
unclassified_below_zero
one_to_three_data_point_above_zero
one_to_three_data_point_below_zero

mon_decr_above_zero
mon_incr_above_zero
inANDdecr_above_zero

mon_decr_below_zero
mon_incr_below_zero
inANDdecr_below_zero

unclassified_below_zero_and_one_to_three_data_point_above_zero
unclassified_below_zero_and_mon_decr_above_zero
unclassified_below_zero_and_mon_incr_above_zero
unclassified_below_zero_and_inANDdecr_above_zero

one_to_three_data_point_below_zero_and_unclassified_above_zero
inANDdecr_below_zero_and_unclassified_above_zero
mon_decr_below_zero_and_unclassified_above_zero
mon_incr_below_zero_and_unclassified_above_zero

flat_below_zero_and_unclassified_above_zero
unclassified_below_zero_and_flat_above_zero
inANDdecr_and_stable_below_zero_and_unclassified_above_zero
mon_decr_and_stable_below_zero_and_unclassified_above_zero
mon_incr_and_stable_below_zero_and_unclassified_above_zero

mon_decr_and_stable_below_zero_and_unclassified_above_zero
mon_incr_and_stable_below_zero_and_unclassified_above_zero
inANDdecr_and_stable_below_zero_and_unclassified_above_zero
unclassified_below_zero_and_unclassified_above_zero

unclassified_below_zero_and_mon_decr_and_stable_above_zero
unclassified_below_zero_and_mon_incr_and_stable_above_zero
unclassified_below_zero_and_inANDdecr_and_stable_above_zero


[]

In [147]:
len(flat_below_zero_and_flat_above_zero)+ len(one_to_three_data_point_below_zero_and_flat_above_zero)+ len(one_to_three_data_point_below_zero_and_one_to_three_data_point_above_zero)+ len(flat_below_zero_and_one_to_three_data_point_above_zero)+ len(one_at_zero)+len(one)+len(zero)+len(mon_decr_and_stable_above_zero)+len(mon_incr_and_stable_above_zero)+len(inANDdecr_and_stable_above_zero)+len(mon_decr_and_stable_below_zero)+len(mon_incr_and_stable_below_zero)+len(inANDdecr_and_stable_below_zero)+len(flat_above_zero)+len(flat_below_zero)+len(mon_decr_above_zero)+len(mon_incr_above_zero)+len(inANDdecr_above_zero)+len(mon_decr_below_zero)+len(mon_incr_below_zero)+len(inANDdecr_below_zero)+len(decr_and_stable_below_zero_and_one_to_three_data_point_above_zero)+len(incr_and_stable_below_zero_and_one_to_three_data_point_above_zero)+len(inANDdecr_and_stable_below_zero_and_one_to_three_data_point_above_zero)+len(decr_and_stable_below_zero_and_mon_decr_above_zero)+len(incr_and_stable_below_zero_and_mon_decr_above_zero)+len(inANDdecr_and_stable_below_zero_and_mon_decr_above_zero)+len(decr_and_stable_below_zero_and_mon_incr_above_zero)+len(incr_and_stable_below_zero_and_mon_incr_above_zero)+len(inANDdecr_and_stable_below_zero_and_mon_incr_above_zero)+len(decr_and_stable_below_zero_and_inANDdecr_above_zero)+len(incr_and_stable_below_zero_and_inANDdecr_above_zero)+len(inANDdecr_and_stable_below_zero_and_inANDdecr_above_zero)+len(one_to_three_data_point_below_zero_and_decr_and_stable_above_zero)+len(one_to_three_data_point_below_zero_and_incr_and_stable_above_zero)+len(one_to_three_data_point_below_zero_and_inANDdecr_and_stable_above_zero)+len(one_to_three_data_point_below_zero_and_mon_decr_above_zero)+len(one_to_three_data_point_below_zero_and_mon_incr_above_zero)+len(one_to_three_data_point_below_zero_and_inANDdecr_above_zero)+len(inANDdecr_below_zero_and_mon_decr_above_zero)+len(mon_decr_below_zero_and_mon_decr_above_zero)+len(mon_incr_below_zero_and_mon_decr_above_zero)+len(inANDdecr_below_zero_and_mon_incr_above_zero)+len(mon_decr_below_zero_and_mon_incr_above_zero)+len(mon_incr_below_zero_and_mon_incr_above_zero)+len(inANDdecr_below_zero_and_inANDdecr_above_zero)+len(mon_decr_below_zero_and_inANDdecr_above_zero)+len(mon_incr_below_zero_and_inANDdecr_above_zero)+len(inANDdecr_below_zero_and_mon_decr_and_stable_above_zero)+len(inANDdecr_below_zero_and_mon_incr_and_stable_above_zero)+len(inANDdecr_below_zero_and_inANDdecr_and_stable_above_zero)+len(mon_decr_below_zero_and_inANDdecr_and_stable_above_zero)+len(mon_decr_below_zero_and_mon_decr_and_stable_above_zero)+len(mon_decr_below_zero_and_mon_incr_and_stable_above_zero)+len(mon_incr_below_zero_and_inANDdecr_and_stable_above_zero)+len(mon_incr_below_zero_and_mon_decr_and_stable_above_zero)+len(mon_incr_below_zero_and_mon_incr_and_stable_above_zero)+len(inANDdecr_below_zero_and_one_to_three_data_point_above_zero)+len(mon_decr_below_zero_and_one_to_three_data_point_above_zero)+len(mon_incr_below_zero_and_one_to_three_data_point_above_zero)+len(flat_below_zero_and_inANDdecr_above_zero)+len(flat_below_zero_and_mon_decr_above_zero)+len(flat_below_zero_and_mon_incr_above_zero)+len(flat_below_zero_and_inANDdecr_and_stable_above_zero)+len(flat_below_zero_and_mon_decr_and_stable_above_zero)+len(flat_below_zero_and_mon_incr_and_stable_above_zero)+len(inANDdecr_and_stable_below_zero_and_flat_above_zero)+len(mon_decr_and_stable_below_zero_and_flat_above_zero)+len(mon_incr_and_stable_below_zero_and_flat_above_zero)+len(inANDdecr_below_zero_and_flat_above_zero)+len(mon_decr_below_zero_and_flat_above_zero)+len(mon_incr_below_zero_and_flat_above_zero)+len(inANDdecr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero)+len(inANDdecr_and_stable_below_zero_and_mon_decr_and_stable_above_zero)+len(inANDdecr_and_stable_below_zero_and_mon_incr_and_stable_above_zero)+len(mon_decr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero)+len(mon_decr_and_stable_below_zero_and_mon_decr_and_stable_above_zero)+len(mon_decr_and_stable_below_zero_and_mon_incr_and_stable_above_zero)+len(mon_incr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero)+len(mon_incr_and_stable_below_zero_and_mon_decr_and_stable_above_zero)+len(mon_incr_and_stable_below_zero_and_mon_incr_and_stable_above_zero)+len(unclassified_above_zero)+len(unclassified_below_zero)+len(one_to_three_data_point_below_zero)+len(one_to_three_data_point_above_zero)+len(unclassified_below_zero_and_one_to_three_data_point_above_zero)+len(unclassified_below_zero_and_mon_decr_above_zero)+len(unclassified_below_zero_and_mon_incr_above_zero)+len(unclassified_below_zero_and_inANDdecr_above_zero)+len(one_to_three_data_point_below_zero_and_unclassified_above_zero)+len(inANDdecr_below_zero_and_unclassified_above_zero)+len(mon_decr_below_zero_and_unclassified_above_zero)+len(mon_incr_below_zero_and_unclassified_above_zero)+len(flat_below_zero_and_unclassified_above_zero)+len(unclassified_below_zero_and_flat_above_zero)+len(inANDdecr_and_stable_below_zero_and_unclassified_above_zero)+len(mon_decr_and_stable_below_zero_and_unclassified_above_zero)+len(mon_incr_and_stable_below_zero_and_unclassified_above_zero)+len(unclassified_below_zero_and_unclassified_above_zero)+len(unclassified_below_zero_and_mon_decr_and_stable_above_zero)+len(unclassified_below_zero_and_mon_incr_and_stable_above_zero)+len(unclassified_below_zero_and_inANDdecr_and_stable_above_zero)


2726

In [148]:
lt= list(flat_below_zero_and_flat_above_zero)+ list(one_to_three_data_point_below_zero_and_flat_above_zero)+ list(one_to_three_data_point_below_zero_and_one_to_three_data_point_above_zero)+ list(flat_below_zero_and_one_to_three_data_point_above_zero)+ list(one_at_zero)+list(one)+list(zero)+list(mon_decr_and_stable_above_zero)+list(mon_incr_and_stable_above_zero)+list(inANDdecr_and_stable_above_zero)+list(mon_decr_and_stable_below_zero)+list(mon_incr_and_stable_below_zero)+list(inANDdecr_and_stable_below_zero)+list(flat_above_zero)+list(flat_below_zero)+list(mon_decr_above_zero)+list(mon_incr_above_zero)+list(inANDdecr_above_zero)+list(mon_decr_below_zero)+list(mon_incr_below_zero)+list(inANDdecr_below_zero)+list(decr_and_stable_below_zero_and_one_to_three_data_point_above_zero)+list(incr_and_stable_below_zero_and_one_to_three_data_point_above_zero)+list(inANDdecr_and_stable_below_zero_and_one_to_three_data_point_above_zero)+list(decr_and_stable_below_zero_and_mon_decr_above_zero)+list(incr_and_stable_below_zero_and_mon_decr_above_zero)+list(inANDdecr_and_stable_below_zero_and_mon_decr_above_zero)+list(decr_and_stable_below_zero_and_mon_incr_above_zero)+list(incr_and_stable_below_zero_and_mon_incr_above_zero)+list(inANDdecr_and_stable_below_zero_and_mon_incr_above_zero)+list(decr_and_stable_below_zero_and_inANDdecr_above_zero)+list(incr_and_stable_below_zero_and_inANDdecr_above_zero)+list(inANDdecr_and_stable_below_zero_and_inANDdecr_above_zero)+list(one_to_three_data_point_below_zero_and_decr_and_stable_above_zero)+list(one_to_three_data_point_below_zero_and_incr_and_stable_above_zero)+list(one_to_three_data_point_below_zero_and_inANDdecr_and_stable_above_zero)+list(one_to_three_data_point_below_zero_and_mon_decr_above_zero)+list(one_to_three_data_point_below_zero_and_mon_incr_above_zero)+list(one_to_three_data_point_below_zero_and_inANDdecr_above_zero)+list(inANDdecr_below_zero_and_mon_decr_above_zero)+list(mon_decr_below_zero_and_mon_decr_above_zero)+list(mon_incr_below_zero_and_mon_decr_above_zero)+list(inANDdecr_below_zero_and_mon_incr_above_zero)+list(mon_decr_below_zero_and_mon_incr_above_zero)+list(mon_incr_below_zero_and_mon_incr_above_zero)+list(inANDdecr_below_zero_and_inANDdecr_above_zero)+list(mon_decr_below_zero_and_inANDdecr_above_zero)+list(mon_incr_below_zero_and_inANDdecr_above_zero)+list(inANDdecr_below_zero_and_mon_decr_and_stable_above_zero)+list(inANDdecr_below_zero_and_mon_incr_and_stable_above_zero)+list(inANDdecr_below_zero_and_inANDdecr_and_stable_above_zero)+list(mon_decr_below_zero_and_inANDdecr_and_stable_above_zero)+list(mon_decr_below_zero_and_mon_decr_and_stable_above_zero)+list(mon_decr_below_zero_and_mon_incr_and_stable_above_zero)+list(mon_incr_below_zero_and_inANDdecr_and_stable_above_zero)+list(mon_incr_below_zero_and_mon_decr_and_stable_above_zero)+list(mon_incr_below_zero_and_mon_incr_and_stable_above_zero)+list(inANDdecr_below_zero_and_one_to_three_data_point_above_zero)+list(mon_decr_below_zero_and_one_to_three_data_point_above_zero)+list(mon_incr_below_zero_and_one_to_three_data_point_above_zero)+list(flat_below_zero_and_inANDdecr_above_zero)+list(flat_below_zero_and_mon_decr_above_zero)+list(flat_below_zero_and_mon_incr_above_zero)+list(flat_below_zero_and_inANDdecr_and_stable_above_zero)+list(flat_below_zero_and_mon_decr_and_stable_above_zero)+list(flat_below_zero_and_mon_incr_and_stable_above_zero)+list(inANDdecr_and_stable_below_zero_and_flat_above_zero)+list(mon_decr_and_stable_below_zero_and_flat_above_zero)+list(mon_incr_and_stable_below_zero_and_flat_above_zero)+list(inANDdecr_below_zero_and_flat_above_zero)+list(mon_decr_below_zero_and_flat_above_zero)+list(mon_incr_below_zero_and_flat_above_zero)+list(inANDdecr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero)+list(inANDdecr_and_stable_below_zero_and_mon_decr_and_stable_above_zero)+list(inANDdecr_and_stable_below_zero_and_mon_incr_and_stable_above_zero)+list(mon_decr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero)+list(mon_decr_and_stable_below_zero_and_mon_decr_and_stable_above_zero)+list(mon_decr_and_stable_below_zero_and_mon_incr_and_stable_above_zero)+list(mon_incr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero)+list(mon_incr_and_stable_below_zero_and_mon_decr_and_stable_above_zero)+list(mon_incr_and_stable_below_zero_and_mon_incr_and_stable_above_zero)+list(unclassified_above_zero)+list(unclassified_below_zero)+list(one_to_three_data_point_below_zero)+list(one_to_three_data_point_above_zero)+list(unclassified_below_zero_and_one_to_three_data_point_above_zero)+list(unclassified_below_zero_and_mon_decr_above_zero)+list(unclassified_below_zero_and_mon_incr_above_zero)+list(unclassified_below_zero_and_inANDdecr_above_zero)+list(one_to_three_data_point_below_zero_and_unclassified_above_zero)+list(inANDdecr_below_zero_and_unclassified_above_zero)+list(mon_decr_below_zero_and_unclassified_above_zero)+list(mon_incr_below_zero_and_unclassified_above_zero)+list(flat_below_zero_and_unclassified_above_zero)+list(unclassified_below_zero_and_flat_above_zero)+list(inANDdecr_and_stable_below_zero_and_unclassified_above_zero)+list(mon_decr_and_stable_below_zero_and_unclassified_above_zero)+list(mon_incr_and_stable_below_zero_and_unclassified_above_zero)+list(mon_decr_and_stable_below_zero_and_unclassified_above_zero)+list(mon_incr_and_stable_below_zero_and_unclassified_above_zero)+list(inANDdecr_and_stable_below_zero_and_unclassified_above_zero)+list(unclassified_below_zero_and_unclassified_above_zero)+list(unclassified_below_zero_and_mon_decr_and_stable_above_zero)+list(unclassified_below_zero_and_mon_incr_and_stable_above_zero)+list(unclassified_below_zero_and_inANDdecr_and_stable_above_zero)


In [149]:
l1=[]
for i in lt:
    if i not in l1:
        l1.append(i)
    else:
        print(i,end=' ')
        

In [150]:
csvFile = pd.read_csv('all reactions.csv')
csvFile["Reaction names"]

rl= list(csvFile["Reaction names"])

set1= set(rl)
set2= set(lt)
set3= set1-set2
set3


set()

In [151]:
len(diff_indecr_above_zero)+len(diff_indecr_below_zero)


884

In [152]:
len(mon_decr_above_zero)+len(mon_incr_above_zero)+len(inANDdecr_above_zero)+len(mon_decr_below_zero)+len(mon_incr_below_zero)+len(inANDdecr_below_zero)+len(unclassified_above_zero)+len(unclassified_below_zero)


884

In [153]:
lt= list(mon_decr_above_zero)+list(mon_incr_above_zero)+list(inANDdecr_above_zero)+list(mon_decr_below_zero)+list(mon_incr_below_zero)+list(inANDdecr_below_zero)+list(unclassified_above_zero)+list(unclassified_below_zero)


In [154]:
l1=[]
for i in lt:
    if i not in l1:
        l1.append(i)
    else:
        print(i,end=' ')
        